In [ ]:
# Import các thư viện cần thiết
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModel
import os
import json
import glob

# Train Dataset - Mô hình Fine-tuned

## Trích xuất hidden states theo từng động từ (GramVar) và lưu thành file `.pt`

- **Mục đích**:  
  - Tải mô hình BioBERT đã fine-tune và tokenizer.  
  - Duyệt toàn bộ **tập train** (GramVar) theo **từng động từ** (mỗi file JSON = một động từ).  
  - Với **mỗi câu**, trích xuất **toàn bộ hidden states** (từng lớp của mô hình) và lưu thành các file `.pt`.  
  - Tổ chức kết quả theo cấu trúc thư mục: `.../Representation Vector/Train/Finetuned/train_vectors_finetuned_gramvar/<verb_name>/sentence_<i>.pt`.

- **Quy trình**

1. **Thiết lập & tải mô hình**  
     - Khai báo: `final_model_path`, `drive_base_path`, `train_data_dir`, `train_output_base_dir`.  
     - Tải `tokenizer` (BioBERT base) và **mô hình đã fine-tuned** từ `final_model_path`.  

In [ ]:
# Đường dẫn đến thư mục chứa mô hình tốt nhất của bạn trên Google Drive
final_model_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Finetuned_Models/biobert-srl-best-model'
# Đường dẫn thư mục gốc trên Google Drive để làm việc
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'

# Đường dẫn đến thư mục chứa các file train theo động từ
train_data_dir = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Clean_Dataset/Corpus/Split_GramVar/Train'

# Đường dẫn đến thư mục cha để lưu các vector của tập train
train_output_base_dir = os.path.join(drive_base_path, 'Representation Vector/Train/Finetuned/train_vectors_finetuned_gramvar')

# Tải tokenizer và mô hình
print("Đang tải tokenizer và mô hình fine-tuned...")
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.2")
model_ft = AutoModelForTokenClassification.from_pretrained(final_model_path)
print("Đã tải xong tokenizer và mô hình fine-tuned.")

Đang tải tokenizer và mô hình fine-tuned...
Đã tải xong tokenizer và mô hình fine-tuned.


2. **Hàm cốt lõi**  

 - `get_hidden_states(model, tokenizer, text)`  
       - Tokenize câu, chạy mô hình với `output_hidden_states=True`.  
       - Trả về `hidden_states` (tuple các lớp) và `tokens`.  

In [ ]:
def get_hidden_states(model, tokenizer, text):
    """
    Trích xuất các trạng thái ẩn (vector biểu diễn) từ mỗi lớp của mô hình.
    """
    inputs = tokenizer(text, return_tensors="pt")
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    return outputs.hidden_states, tokens

 - `process_and_save_per_verb(verb_name, sentences, model, tokenizer, output_dir)`  
       - Dùng cho **một động từ**: duyệt từng câu, trích xuất & lưu lần lượt `sentence_0.pt`, `sentence_1.pt`, …

In [ ]:
def process_and_save_per_verb(verb_name, sentences, model, tokenizer, output_dir):
    """
    Xử lý một danh sách câu cho một động từ và lưu ra file riêng.
    """
    print(f"\n--- Xử lý cho động từ: {verb_name} ---")
    os.makedirs(output_dir, exist_ok=True)

    total_sentences = len(sentences)
    for i, sentence in enumerate(sentences):
        if not sentence.strip():
            continue

        print(f"Đang xử lý câu {i+1}/{total_sentences}...")

        hidden_states, tokens = get_hidden_states(model, tokenizer, sentence)
        vectors_to_save = {}
        for j, layer_rep in enumerate(hidden_states):
            vectors_to_save[f"layer_{j}"] = layer_rep.squeeze(0)

        file_name = f"sentence_{i}.pt"
        output_path = os.path.join(output_dir, file_name)

        torch.save(vectors_to_save, output_path)

3. **Thực thi**   
  - Dùng `glob` tìm tất cả file `*.json` trong `Split_GramVar/Train`.  
  - Từ tên file (ví dụ `begin_1_train_set.json`) suy ra `verb_name = begin_1`.  
  - Đọc JSON, lấy danh sách `text` (lọc rỗng), tạo thư mục `train_vectors_finetuned_gramvar/<verb_name>/`.  
  - Gọi `process_and_save_per_verb` để lưu toàn bộ hidden states cho từng câu.


In [ ]:
if __name__ == "__main__":

    # Xử lý toàn bộ tập dữ liệu train theo từng động từ
    try:
        # Tìm tất cả các file .json trong thư mục train
        json_files = glob.glob(os.path.join(train_data_dir, '*.json'))

        if not json_files:
            print(f"LỖI: Không tìm thấy file .json nào trong thư mục: {train_data_dir}")
        else:
            print(f"\nTìm thấy {len(json_files)} file .json để xử lý.")

        # Lặp qua từng file json
        for file_path in json_files:
            # Lấy tên file gốc (ví dụ: 'begin_1_train_set.json')
            base_name = os.path.basename(file_path)
            # Tách các phần bằng dấu '_'
            parts = base_name.replace('_train_set.json', '').split('_')
            # Nối lại các phần đầu để tạo tên động từ (ví dụ: 'begin_1')
            verb_name = "_".join(parts)

            # Đọc dữ liệu từ file JSON
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # Trích xuất các câu từ trường 'text'
            sentences = [item['text'] for item in data if 'text' in item and item['text'].strip()]

            # Tạo thư mục output riêng cho động từ này
            verb_output_dir = os.path.join(train_output_base_dir, verb_name)

            # Xử lý và lưu các câu cho động từ hiện tại
            process_and_save_per_verb(verb_name, sentences, model_ft, tokenizer, verb_output_dir)

    except FileNotFoundError:
        print(f"\nLỖI: Không tìm thấy thư mục dữ liệu train tại '{train_data_dir}'.")
    except Exception as e:
        print(f"\nĐã xảy ra lỗi không mong muốn: {e}")

- **Kết quả**:  
  - Sinh cấu trúc output theo từng động từ, ví dụ:  
    ```
    .../train_vectors_finetuned_gramvar/
      └── begin_1/
          ├── sentence_0.pt
          ├── sentence_1.pt
          └── ...
    ```  
  - Mỗi file `.pt` là một dict:  
    - Key: `layer_0`, `layer_1`, … (embedding + các lớp Transformer).  
    - Value: tensor kích thước `[num_tokens, hidden_size]` (với BioBERT base: `hidden_size = 768`).  

- **Lưu ý**:  
  - Đảm bảo `final_model_path` và `train_data_dir` trỏ đúng tới Drive.  
  - Các câu rỗng sẽ bị bỏ qua (`.strip()`).  
  - Nếu muốn tái lập thứ tự xử lý/ghi log, có thể thiết lập seed trước khi duyệt.  
  - Các file JSON đầu vào cần có trường `text` trong từng phần tử.

## Tạo file index cho từng động từ (GramVar)

- **Mục đích**:  
  - Tự động sinh các **file index `.json`** cho từng động từ trong tập **Train**.  
  - Mỗi file index liệt kê **toàn bộ câu** và **đường dẫn file vector `.pt`** tương ứng đã/ sẽ được trích xuất.  
  - Hỗ trợ tra cứu nhanh: từ (động từ, chỉ số câu) → mở đúng file vector để phân tích.

- **Quy trình**

  1. **Thiết lập đường dẫn**  
     - `train_data_dir`: nơi chứa các file train theo động từ (ví dụ `begin_1_train_set.json`).  
     - `train_output_base_dir`: thư mục cha chứa các vector đã lưu theo động từ (mỗi câu = một file `.pt`).  
     - `output_index_dir`: nơi lưu **các file index** (mỗi động từ một file JSON).  

In [ ]:
# Đường dẫn thư mục gốc trên Google Drive để làm việc
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'

# Đường dẫn đến thư mục chứa các file train gốc (.json)
train_data_dir = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Clean_Dataset/Corpus/Split_GramVar/Train'

# Đường dẫn đến thư mục cha chứa các vector đã được xử lý
train_output_base_dir = os.path.join(drive_base_path, 'Representation Vector/Train/Finetuned/train_vectors_finetuned_gramvar')

# Đường dẫn đến thư mục để lưu các file index JSON riêng lẻ
output_index_dir = os.path.join(drive_base_path, 'Representation Vector/Train_Index/Finetuned/verb_indexes_finetuned_gramvar_train')

 2. **Hàm `create_sentence_index_per_verb()`**  
     - Quét tất cả `*.json` trong `train_data_dir`.  
     - Suy ra `verb_name` từ tên file (vd: `begin_1_train_set.json` → `begin_1`).  
     - Đọc dữ liệu JSON, lấy danh sách câu từ trường `text` (lọc rỗng).  
     - Với mỗi câu `i`, tạo entry:  
       - `sentence_text`: câu gốc  
       - `verb_name`: tên động từ  
       - `vector_file_path`: đường dẫn dự kiến đến file vector (vd: `<train_vectors_finetuned_gramvar>/<verb_name>/sentence_i.pt`)  
     - Ghi toàn bộ entry ra file `output_index_dir/<verb_name>.json`.  

In [ ]:
def create_sentence_index_per_verb():
    """
    Quét qua các file dữ liệu train và tạo ra một file index .json riêng
    cho mỗi động từ.
    """

    print("Bắt đầu quá trình tạo file index cho từng động từ...")

    # Tạo thư mục chứa các file index nếu nó chưa tồn tại
    os.makedirs(output_index_dir, exist_ok=True)

    try:
        # Tìm tất cả các file .json trong thư mục train gốc
        json_files = glob.glob(os.path.join(train_data_dir, '*.json'))

        if not json_files:
            print(f"LỖI: Không tìm thấy file .json nào trong thư mục: {train_data_dir}")
            return

        print(f"Tìm thấy {len(json_files)} file .json để tạo index.")

        # Lặp qua từng file json
        for file_path in json_files:
            # Lấy tên động từ từ tên file (ví dụ: 'begin_1')
            base_name = os.path.basename(file_path)
            verb_name = "_".join(base_name.replace('_train_set.json', '').split('_'))

            print(f"\n--- Đang xử lý cho động từ: {verb_name} ---")

            # Khởi tạo một danh sách để chứa index cho động từ hiện tại
            verb_index_list = []

            # Đọc dữ liệu từ file JSON
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # Trích xuất các câu từ trường 'text'
            sentences = [item['text'] for item in data if 'text' in item and item['text'].strip()]

            # Lặp qua từng câu để tạo entry cho file index
            for i, sentence in enumerate(sentences):
                # Xây dựng đường dẫn dự kiến đến file vector .pt tương ứng
                vector_file_name = f"sentence_{i}.pt"
                vector_file_path = os.path.join(train_output_base_dir, verb_name, vector_file_name)

                # Tạo một entry cho file index
                index_entry = {
                    "sentence_text": sentence,
                    "verb_name": verb_name,
                    "vector_file_path": vector_file_path
                }
                verb_index_list.append(index_entry)

            # Sau khi xử lý xong các câu của một động từ, lưu ra file JSON riêng
            if verb_index_list:
                output_verb_index_file = os.path.join(output_index_dir, f"{verb_name}.json")
                print(f"Đang lưu file index cho '{verb_name}' vào: {output_verb_index_file}")
                with open(output_verb_index_file, 'w', encoding='utf-8') as f:
                    # Lưu danh sách các entry, không phải dictionary
                    json.dump(verb_index_list, f, ensure_ascii=False, indent=4)
                print(f"Đã tạo file index cho {len(sentences)} câu của động từ '{verb_name}'.")

        print("\nQUÁ TRÌNH TẠO FILE INDEX ĐÃ HOÀN TẤT!")

    except FileNotFoundError:
        print(f"\nLỖI: Không tìm thấy thư mục dữ liệu train tại '{train_data_dir}'.")
    except Exception as e:
        print(f"\nĐã xảy ra lỗi không mong muốn: {e}")

3. **Thực thi**  
     - Gọi `create_sentence_index_per_verb()` trong `if __name__ == "__main__":`.  
     - In log tiến trình: số file train được quét, từng động từ đang xử lý, nơi lưu file index, v.v.

In [ ]:
if __name__ == "__main__":
    create_sentence_index_per_verb()

Bắt đầu quá trình tạo file index cho từng động từ...
Tìm thấy 35 file .json để tạo index.

--- Đang xử lý cho động từ: begin_1 ---
Đang lưu file index cho 'begin_1' vào: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/verb_indexes_gramvar_finetuned/begin_1.json
Đã tạo file index cho 96 câu của động từ 'begin_1'.

--- Đang xử lý cho động từ: begin_2 ---
Đang lưu file index cho 'begin_2' vào: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/verb_indexes_gramvar_finetuned/begin_2.json
Đã tạo file index cho 137 câu của động từ 'begin_2'.

--- Đang xử lý cho động từ: translate_2 ---
Đang lưu file index cho 'translate_2' vào: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/verb_indexes_gramvar_finetuned/translate_2.json
Đã tạo file index cho 149 câu của động từ 'translate_2'.

--- Đang xử lý cho động từ: transform_1 ---
Đang lưu file index cho 'transform_1' vào: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/verb_indexes_gramvar_finetuned/trans

- **Kết quả**:  
  - Thư mục `verb_indexes_finetuned_gramvar_train/` chứa một loạt file JSON, mỗi file ứng với **một động từ**:  
    ```
    .../verb_indexes_finetuned_gramvar_train
      ├── begin_1.json
      ├── block.json
      └── ...
    ```  
  - Mỗi file là **một danh sách** các entry có dạng:
    ```json
    {
      "sentence_text": "...",
      "verb_name": "begin_1",
      "vector_file_path": "/.../train_vectors_finetuned_gramvar/begin_1/sentence_0.pt"
    }
    ```  
  - Sẵn sàng sử dụng cho các bước phân tích sau (ví dụ: load vector theo `verb` + `sentence_index`).

- **Lưu ý**:  
  - Đảm bảo cấu trúc tên file train kết thúc bằng `_train_set.json`.  
  - Các vector `.pt` cần được lưu theo quy ước cùng thư mục `train_vectors_finetuned_gramvar/<verb_name>/sentence_i.pt`.  
  - Nếu chưa tạo vector, index vẫn có thể sinh ra trước (đường dẫn “dự kiến”); khi trích xuất xong sẽ khớp đúng.

## Trích xuất hidden states theo từng động từ (ParaVE) và lưu thành file `.pt`

- **Mục đích**:  
  - Tải mô hình BioBERT đã fine-tune và tokenizer.  
  - Duyệt **tập train của ParaVE** theo **từng động từ** (mỗi file JSON ứng với một động từ).  
  - Với **mỗi câu**, trích xuất **toàn bộ hidden states** (mọi lớp của mô hình) và lưu thành các file `.pt`.  
  - Tổ chức kết quả theo cấu trúc thư mục: `.../Representation Vector/Train/Finetunedtrain_vectors_finetuned_parave/<verb_name>/sentence_<i>.pt`.

- **Quy trình**

  1. **Thiết lập & tải mô hình**  
     - Khai báo: `final_model_path`, `drive_base_path`, `train_data_dir` (trỏ vào `Split_ParaVE/Train`), `train_output_base_dir`.  
     - Tải `tokenizer` (BioBERT base) và **mô hình đã fine-tuned** từ `final_model_path`.  

In [ ]:
# Đường dẫn đến thư mục chứa mô hình tốt nhất của bạn trên Google Drive
final_model_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Finetuned_Models/biobert-srl-best-model'
# Đường dẫn thư mục gốc trên Google Drive để làm việc
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'

#  Đường dẫn đến thư mục chứa các file train theo động từ
train_data_dir = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Clean_Dataset/Corpus/Split_ParaVE/Train'

# Đường dẫn đến thư mục cha để lưu các vector của tập train
train_output_base_dir = os.path.join(drive_base_path, 'Representation Vector/Train/Finetuned/train_vectors_finetuned_parave')

# Tải tokenizer và mô hình
print("Đang tải tokenizer và mô hình fine-tuned...")
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.2")
model_ft = AutoModelForTokenClassification.from_pretrained(final_model_path)
print("Đã tải xong tokenizer và mô hình fine-tuned.")

 2. **Hàm cốt lõi**  

- `get_hidden_states(model, tokenizer, text)`  
       - Tokenize câu, gọi mô hình với `output_hidden_states=True`.  
       - Trả về `hidden_states` (tuple các lớp) và `tokens`.

In [ ]:
def get_hidden_states(model, tokenizer, text):
    """
    Trích xuất các trạng thái ẩn (vector biểu diễn) từ mỗi lớp của mô hình.
    """
    inputs = tokenizer(text, return_tensors="pt")
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    return outputs.hidden_states, tokens

- `process_and_save_per_verb(verb_name, sentences, model, tokenizer, output_dir)`  
       - Dùng cho **một động từ**: duyệt từng câu, trích xuất & lưu lần lượt `sentence_0.pt`, `sentence_1.pt`, …

In [ ]:
def process_and_save_per_verb(verb_name, sentences, model, tokenizer, output_dir):
    """
    Xử lý một danh sách câu cho một động từ và lưu ra file riêng.
    """
    print(f"\n--- Xử lý cho động từ: {verb_name} ---")
    os.makedirs(output_dir, exist_ok=True)

    total_sentences = len(sentences)
    for i, sentence in enumerate(sentences):
        if not sentence.strip():
            continue

        print(f"  Đang xử lý câu {i+1}/{total_sentences}...")

        hidden_states, tokens = get_hidden_states(model, tokenizer, sentence)
        vectors_to_save = {}
        for j, layer_rep in enumerate(hidden_states):
            vectors_to_save[f"layer_{j}"] = layer_rep.squeeze(0)

        file_name = f"sentence_{i}.pt"
        output_path = os.path.join(output_dir, file_name)

        torch.save(vectors_to_save, output_path)

    print(f"Đã xử lý xong {total_sentences} câu cho động từ '{verb_name}'.")

3. **Thực thi**  
- Dùng `glob` tìm tất cả file `*.json` trong `Split_ParaVE/Train`.  
- Từ tên file (vd: `begin_1_train_set.json`) suy ra `verb_name = begin_1`.  
- Đọc JSON, lấy danh sách `text` (lọc rỗng), tạo thư mục `train_vectors_finetuned_parave/<verb_name>/`.  
- Gọi `process_and_save_per_verb` để lưu hidden states cho toàn bộ câu của động từ đó.

In [ ]:
if __name__ == "__main__":

    # Xử lý toàn bộ tập dữ liệu train theo từng động từ
    try:
        # Tìm tất cả các file .json trong thư mục train
        json_files = glob.glob(os.path.join(train_data_dir, '*.json'))

        if not json_files:
            print(f"LỖI: Không tìm thấy file .json nào trong thư mục: {train_data_dir}")
        else:
            print(f"\nTìm thấy {len(json_files)} file .json để xử lý.")

        # Lặp qua từng file json
        for file_path in json_files:
            # Lấy tên file gốc (ví dụ: 'begin_1_train_set.json')
            base_name = os.path.basename(file_path)
            # Tách các phần bằng dấu '_'
            parts = base_name.replace('_train_set.json', '').split('_')
            # Nối lại các phần đầu để tạo tên động từ (ví dụ: 'begin_1')
            verb_name = "_".join(parts)

            # Đọc dữ liệu từ file JSON
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # Trích xuất các câu từ trường 'text'
            sentences = [item['text'] for item in data if 'text' in item and item['text'].strip()]

            # Tạo thư mục output riêng cho động từ này
            verb_output_dir = os.path.join(train_output_base_dir, verb_name)

            # Xử lý và lưu các câu cho động từ hiện tại
            process_and_save_per_verb(verb_name, sentences, model_ft, tokenizer, verb_output_dir)

    except FileNotFoundError:
        print(f"\nLỖI: Không tìm thấy thư mục dữ liệu train tại '{train_data_dir}'.")
    except Exception as e:
        print(f"\nĐã xảy ra lỗi không mong muốn: {e}")



Đang tải tokenizer và mô hình fine-tuned...
Đã tải xong tokenizer và mô hình fine-tuned.

--- Xử lý câu test ---
Câu: 'the net load of the variant subunit may result in a variant electrophoretic subunit LDH-B .'
ĐÃ LƯU THÀNH CÔNG! Vector của câu test được lưu tại: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/test_sentence_vector.pt

Tìm thấy 35 file .json để xử lý.

--- Xử lý cho động từ: catalyse ---
  Đang xử lý câu 1/43...
  Đang xử lý câu 2/43...
  Đang xử lý câu 3/43...
  Đang xử lý câu 4/43...
  Đang xử lý câu 5/43...
  Đang xử lý câu 6/43...
  Đang xử lý câu 7/43...
  Đang xử lý câu 8/43...
  Đang xử lý câu 9/43...
  Đang xử lý câu 10/43...
  Đang xử lý câu 11/43...
  Đang xử lý câu 12/43...
  Đang xử lý câu 13/43...
  Đang xử lý câu 14/43...
  Đang xử lý câu 15/43...
  Đang xử lý câu 16/43...
  Đang xử lý câu 17/43...
  Đang xử lý câu 18/43...
  Đang xử lý câu 19/43...
  Đang xử lý câu 20/43...
  Đang xử lý câu 21/43...
  Đang xử lý câu 22/43...
  Đang xử lý câu 

- **Kết quả**:  
  - Sinh cấu trúc output theo từng động từ, ví dụ:  
    ```
    .../train_vectors_finetuned_parave/
      └── begin_1/
          ├── sentence_0.pt
          ├── sentence_1.pt
          └── ...
    ```  
  - Mỗi file `.pt` là một dict:  
    - Key: `layer_0`, `layer_1`, … (embedding + các lớp Transformer).  
    - Value: tensor kích thước `[num_tokens, hidden_size]` (BioBERT base: `hidden_size = 768`).

- **Lưu ý**:  
  - Đảm bảo `final_model_path` và `train_data_dir` chính xác (ParaVE, không phải GramVar).  
  - Các câu rỗng được bỏ qua (`.strip()`).  
  - Nếu cần tái lập trình tự xử lý/ghi log, có thể đặt seed trước khi duyệt.  
  - Các file JSON đầu vào cần có trường `text` trong từng phần tử.

## Tạo file index cho từng động từ (ParaVE)

- **Mục đích**:  
  - Sinh các **file index `.json`** cho từng động từ trong **tập Train của ParaVE**.  
  - Mỗi file index liệt kê **câu gốc** và **đường dẫn file vector `.pt`** tương ứng (sẽ/đã được trích xuất).  
  - Hỗ trợ tra cứu nhanh: (động từ, chỉ số câu) → mở đúng file vector để phân tích.

- **Quy trình**

 1. **Thiết lập đường dẫn**  
     - `train_data_dir`: thư mục chứa các file train của ParaVE (mỗi file một động từ, ví dụ `begin_1_train_set.json`).  
     - `train_output_base_dir`: thư mục cha sẽ chứa vector `.pt` theo cấu trúc `<verb_name>/sentence_<i>.pt`.  
     - `output_index_dir`: nơi lưu **các file index** (mỗi động từ một file JSON).

In [ ]:
# Đường dẫn thư mục gốc trên Google Drive để làm việc
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'

# Đường dẫn đến thư mục chứa các file train gốc (.json)
train_data_dir = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Clean_Dataset/Corpus/Split_ParaVE/Train'

# Đường dẫn đến thư mục cha chứa các vector đã được xử lý
train_output_base_dir = os.path.join(drive_base_path, 'Representation Vector/Train/Finetuned/train_vectors_finetuned_parave/')

# Đường dẫn đến thư mục để lưu các file index JSON riêng lẻ
output_index_dir = os.path.join(drive_base_path, 'verb_indexes_finetuned_parave_train')

 2. **Hàm `create_sentence_index_per_verb()`**  
     - Quét tất cả file `*.json` trong `train_data_dir`.  
     - Suy ra `verb_name` từ tên file (vd: `begin_1_train_set.json` → `begin_1`).  
     - Đọc JSON, lấy danh sách các câu từ trường `text` (lọc rỗng).  
     - Với mỗi câu `i`, tạo một entry gồm:  
       - `sentence_text`: câu gốc.  
       - `verb_name`: tên động từ.  
       - `vector_file_path`: đường dẫn dự kiến tới vector `.pt` (vd: `.../verb_indexes_finetuned_parave_train/begin_1/sentence_0.pt`).  
     - Ghi toàn bộ entry ra file `output_index_dir/<verb_name>.json` (định dạng danh sách, `indent=4`).

In [ ]:
def create_sentence_index_per_verb():
    """
    Quét qua các file dữ liệu train và tạo ra một file index .json riêng
    cho mỗi động từ.
    """

    print("Bắt đầu quá trình tạo file index cho từng động từ...")

    # Tạo thư mục chứa các file index nếu nó chưa tồn tại
    os.makedirs(output_index_dir, exist_ok=True)

    try:
        # Tìm tất cả các file .json trong thư mục train gốc
        json_files = glob.glob(os.path.join(train_data_dir, '*.json'))

        if not json_files:
            print(f"LỖI: Không tìm thấy file .json nào trong thư mục: {train_data_dir}")
            return

        print(f"Tìm thấy {len(json_files)} file .json để tạo index.")

        # Lặp qua từng file json
        for file_path in json_files:
            # Lấy tên động từ từ tên file (ví dụ: 'begin_1')
            base_name = os.path.basename(file_path)
            verb_name = "_".join(base_name.replace('_train_set.json', '').split('_'))

            print(f"\n--- Đang xử lý cho động từ: {verb_name} ---")

            # Khởi tạo một danh sách để chứa index cho động từ hiện tại
            verb_index_list = []

            # Đọc dữ liệu từ file JSON
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # Trích xuất các câu từ trường 'text'
            sentences = [item['text'] for item in data if 'text' in item and item['text'].strip()]

            # Lặp qua từng câu để tạo entry cho file index
            for i, sentence in enumerate(sentences):
                # Xây dựng đường dẫn dự kiến đến file vector .pt tương ứng
                vector_file_name = f"sentence_{i}.pt"
                vector_file_path = os.path.join(train_output_base_dir, verb_name, vector_file_name)

                # Tạo một entry cho file index
                index_entry = {
                    "sentence_text": sentence,
                    "verb_name": verb_name,
                    "vector_file_path": vector_file_path
                }
                verb_index_list.append(index_entry)

            # Sau khi xử lý xong các câu của một động từ, lưu ra file JSON riêng
            if verb_index_list:
                output_verb_index_file = os.path.join(output_index_dir, f"{verb_name}.json")
                print(f"Đang lưu file index cho '{verb_name}' vào: {output_verb_index_file}")
                with open(output_verb_index_file, 'w', encoding='utf-8') as f:
                    # Lưu danh sách các entry, không phải dictionary
                    json.dump(verb_index_list, f, ensure_ascii=False, indent=4)
                print(f"Đã tạo file index cho {len(sentences)} câu của động từ '{verb_name}'.")

        print("\nQUÁ TRÌNH TẠO FILE INDEX ĐÃ HOÀN TẤT!")

    except FileNotFoundError:
        print(f"\nLỖI: Không tìm thấy thư mục dữ liệu train tại '{train_data_dir}'.")
    except Exception as e:
        print(f"\nĐã xảy ra lỗi không mong muốn: {e}")

3. **Thực thi**  
     - Gọi `create_sentence_index_per_verb()` trong khối `if __name__ == "__main__":`.  
     - In log tiến trình (số file train, tên động từ đang xử lý, đường dẫn file index đã lưu, v.v.).

In [ ]:
if __name__ == "__main__":
    create_sentence_index_per_verb()

Bắt đầu quá trình tạo file index cho từng động từ...
Tìm thấy 35 file .json để tạo index.

--- Đang xử lý cho động từ: catalyse ---
Đang lưu file index cho 'catalyse' vào: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/verb_indexes_parave_finetuned/catalyse.json
Đã tạo file index cho 43 câu của động từ 'catalyse'.

--- Đang xử lý cho động từ: result ---
Đang lưu file index cho 'result' vào: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/verb_indexes_parave_finetuned/result.json
Đã tạo file index cho 218 câu của động từ 'result'.

--- Đang xử lý cho động từ: develop ---
Đang lưu file index cho 'develop' vào: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/verb_indexes_parave_finetuned/develop.json
Đã tạo file index cho 10 câu của động từ 'develop'.

--- Đang xử lý cho động từ: eliminate ---
Đang lưu file index cho 'eliminate' vào: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/verb_indexes_parave_finetuned/eliminate.json
Đã tạo file ind

- **Kết quả**:  
  - Thư mục `verb_indexes_finetuned_parave_train/` chứa nhiều file JSON, mỗi file ứng với **một động từ**:  
    ```
    .../verb_indexes_finetuned_parave_train/
      ├── begin_1.json
      ├── block.json
      └── ...
    ```
  - Mỗi file là **một danh sách** entry có dạng:
    ```json
    {
      "sentence_text": "...",
      "verb_name": "begin_1",
      "vector_file_path": "/.../train_vectors_finetuned_parave/begin_1/sentence_0.pt"
    }
    ```
  - Sẵn sàng dùng cho các bước phân tích sau: nạp nhanh vector theo `verb_name` + `sentence_index`.

- **Lưu ý**:  
  - Cấu trúc & logic **tương tự GramVar**, chỉ khác **đường dẫn nguồn** (ParaVE) và **thư mục output** (`train_vectors_by_verb_parave`).  
  - Index có thể tạo **trước** khi trích xuất vector; khi vector `.pt` được sinh đúng cấu trúc, đường dẫn trong index sẽ khớp.  

# Train Dataset - Mô hình Pre-trained

## Trích xuất hidden states theo từng động từ (GramVar) và lưu thành file `.pt`

- **Mục đích**
  - Lấy biểu diễn ẩn (hidden states) của **mọi lớp** từ mô hình BioBERT gốc (`dmis-lab/biobert-base-cased-v1.2`) cho **từng câu** trong tập train.
  - Lưu lại dưới dạng file `.pt` theo **từng động từ** để dùng cho các bước thống kê/chuẩn hoá/huấn luyện downstream.

- **Đầu vào**
  - Thư mục JSON của tập train tách theo động từ:  
    `.../Clean_Dataset/Corpus/Split_GramVar/Train/*.json`
  - Mỗi JSON là danh sách các item có khoá `text`.

- **Đầu ra**
  - Cấu trúc thư mục vector:  
    `.../train_vectors_pretrained_gramvar/<verb_name>/sentence_<i>.pt`
  - Mỗi file `.pt` là **dict** có 13 khoá:
    - `layer_0, layer_1, ..., layer_12` → Tensor `[num_tokens, 768]` (biểu diễn của từng token ở lớp tương ứng).

- **Quy trình**
  1. **Tải mô hình & tokenizer pre-trained**
     - Model: `AutoModel.from_pretrained(PRETRAINED_MODEL_NAME)` (không có head phân loại).
     - Tokenizer: `AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)`.
  2. **Đọc dữ liệu** theo từng file động từ trong thư mục `Train/`.
  3. **Lặp từng câu**:
     - Tokenize câu → forward model với `output_hidden_states=True`.
     - Lấy `outputs.hidden_states` (tuple 13 lớp).
     - Tạo dict `{f'layer_{i}': layer_tensor.squeeze(0)}`.
     - Lưu `sentence_<i>.pt` vào thư mục của động từ tương ứng.
  4. **Lặp tới hết** tất cả JSON / tất cả câu.

In [ ]:
# --- 1. THIẾT LẬP CÁC ĐƯỜNG DẪN VÀ TẢI MÔ HÌNH ---

# Đường dẫn thư mục gốc trên Google Drive
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'

# Đường dẫn đến thư mục chứa các file dữ liệu GỐC
train_data_dir = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Clean_Dataset/Corpus/Split_GramVar/Train'

# Đường dẫn đến thư mục MỚI để lưu các vector từ mô hình PRE-TRAINED
output_base_dir = os.path.join(drive_base_path, 'Representation Vector/Train/Pretrained/train_vectors_pretrained_gramvar')
os.makedirs(output_base_dir, exist_ok=True)

# Tải tokenizer và mô hình pre-trained
print("--- Bắt đầu tải mô hình Pre-trained (BioBERT gốc) ---")
try:
    # Tên của mô hình pre-train trên Hugging Face Hub
    PRETRAINED_MODEL_NAME = "dmis-lab/biobert-base-cased-v1.2"

    tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
    # QUAN TRỌNG: Sử dụng AutoModel để tải mô hình gốc, không có lớp phân loại
    model_pt = AutoModel.from_pretrained(PRETRAINED_MODEL_NAME)
    print(" -> Tải thành công tokenizer và mô hình pre-trained!")
except Exception as e:
    print(f"LỖI: Không thể tải mô hình từ Hugging Face. Lỗi: {e}")
    exit()

# --- 2. HÀM XỬ LÝ CỐT LÕI ---

def get_hidden_states_from_pretrained(model, tokenizer, text):
    """
    Trích xuất các trạng thái ẩn (vector biểu diễn) từ mỗi lớp của mô hình pre-trained.
    """
    inputs = tokenizer(text, return_tensors="pt")
    model.eval()
    with torch.no_grad():
        # Yêu cầu mô hình trả về kết quả từ tất cả các lớp
        outputs = model(**inputs, output_hidden_states=True)

    # outputs.hidden_states là một tuple chứa các tensor biểu diễn từ mỗi lớp
    return outputs.hidden_states

# --- 3. THỰC THI CHƯƠNG TRÌNH ---

if __name__ == "__main__":
    print("\n--- Bắt đầu quá trình trích xuất vector từ mô hình Pre-trained ---")

    try:
        # Tìm tất cả các file .json trong thư mục dữ liệu gốc
        json_files = glob.glob(os.path.join(train_data_dir, '*.json'))
        total_files = len(json_files)

        if total_files == 0:
            print(f"LỖI: Không tìm thấy file .json nào trong '{train_data_dir}'.")

        # Lặp qua tất cả các file json gốc
        for file_idx, file_path in enumerate(json_files):
            # Lấy tên động từ từ tên file
            base_name = os.path.basename(file_path)
            verb_name = "_".join(base_name.replace('_train_set.json', '').split('_'))

            print(f"\nĐang xử lý động từ {file_idx + 1}/{total_files}: {verb_name}")

            # Tạo thư mục con tương ứng trong thư mục output
            output_verb_dir = os.path.join(output_base_dir, verb_name)
            os.makedirs(output_verb_dir, exist_ok=True)

            # Đọc dữ liệu từ file JSON gốc
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # Trích xuất các câu từ trường 'text'
            sentences = [item['text'] for item in data if 'text' in item and item['text'].strip()]

            # Lặp qua từng câu
            for i, sentence_text in enumerate(sentences):
                # Trích xuất vector từ mô hình pre-trained
                hidden_states = get_hidden_states_from_pretrained(model_pt, tokenizer, sentence_text)

                # Chuẩn bị dữ liệu để lưu
                vectors_to_save = {}
                for j, layer_rep in enumerate(hidden_states):
                    vectors_to_save[f"layer_{j}"] = layer_rep.squeeze(0)

                # Xây dựng đường dẫn lưu file mới
                file_name = f"sentence_{i}.pt"
                output_path = os.path.join(output_verb_dir, file_name)

                # Lưu file
                torch.save(vectors_to_save, output_path)

            print(f"  -> Đã trích xuất và lưu {len(sentences)} file vector cho động từ '{verb_name}'.")

    except Exception as e:
        print(f"\nĐã xảy ra lỗi không mong muốn: {e}")

    print("\n--- QUÁ TRÌNH TRÍCH XUẤT VECTOR PRE-TRAINED ĐÃ HOÀN TẤT! ---")


- **Lưu ý**
  - Dùng `AutoModel` (không phải `AutoModelForTokenClassification`) để đảm bảo trích xuất **biểu diễn gốc** của BioBERT.
  - `num_tokens` tuỳ vào cách tokenizer tách từ & đặc biệt có token `[CLS]`, `[SEP]`.
  - Nếu chạy trên CPU sẽ chậm; có thể thêm `.to('cuda')` cho model & input nếu có GPU.
  - Thư mục đích `train_vectors_pretrained_gramvar` sẽ được tạo tự động.

## Tạo file index cho từng động từ (GramVar)

- **Mục đích**
  - Đọc các file JSON *train* của **GramVar** (theo từng động từ) để lấy **câu gốc**.
  - Ghép mỗi câu với **đường dẫn vector `.pt`** đã trích từ mô hình BioBERT *pre-trained*.
  - Lưu thành **một file index `.json`/động từ** để tra cứu nhanh cho các bước chuẩn hoá, phân tích hoặc huấn luyện.

- **Đầu vào**
  - Thư mục chứa các file JSON câu gốc (theo động từ):  
    `.../Clean_Dataset/Corpus/Split_GramVar/Train/*.json`
  - Thư mục chứa vector `.pt` đã trích sẵn:  
    `.../train_vectors_pretrained_gramvar/<verb_name>/sentence_<i>.pt`

- **Đầu ra**
  - Thư mục index:  
    `.../verb_indexes_pretrained_gramvar_train/`
  - Mỗi động từ có **1 file** `<verb_name>.json` là **list** các entry:
    ```json
    [
        {
            "sentence_text": "<câu gốc>",
            "verb_name": "<verb_name>",
            "vector_file_path": ".../train_vectors_pretrained_gramvar/<verb_name>/sentence_<i>.pt"
        },
        ...
    ]
    ```

- **Quy trình**
  1. Quét tất cả các file `.json` trong thư mục `Split_GramVar/Train/`.
  2. Suy ra `verb_name` từ tên file (ví dụ: `bind_1_train_set.json` → `bind_1`).
  3. Đọc danh sách các câu (`text`) trong file JSON.
  4. Với từng câu **i**:
     - Ghép đường dẫn tương ứng đến file vector `.pt` đã tạo.
     - Tạo entry gồm `sentence_text`, `verb_name`, `vector_file_path`.
  5. Ghi toàn bộ danh sách entry ra file index `<verb_name>.json` trong thư mục output.

In [ ]:
# Import các thư viện cần thiết
import os
import json
import glob

# --- 1. THIẾT LẬP CÁC ĐƯỜNG DẪN ---

# Đường dẫn thư mục gốc trên Google Drive
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'

# Đường dẫn đến thư mục chứa các file dữ liệu GỐC (nơi lấy câu)
train_data_dir = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Clean_Dataset/Corpus/Split_GramVar/Train'

# Đường dẫn đến thư mục chứa các file vector .pt ĐÃ ĐƯỢC TẠO từ mô hình pre-trained
vectors_base_dir = os.path.join(drive_base_path, 'Representation Vector/Train/Pretrained/train_vectors_pretrained_gramvar')

# Đường dẫn đến thư mục MỚI để lưu các file index
index_output_dir = os.path.join(drive_base_path, 'Representation Vector/Train_Index/Pretrained/verb_indexes_pretrained_gramvar_train')
os.makedirs(index_output_dir, exist_ok=True)


# --- 2. THỰC THI CHƯƠNG TRÌNH TẠO INDEX ---

if __name__ == "__main__":
    print(f"--- Bắt đầu quá trình tạo index cho các vector Pre-trained ---")
    print(f"Đọc dữ liệu gốc từ: {train_data_dir}")
    print(f"Lưu file index vào: {index_output_dir}")

    try:
        # Tìm tất cả các file .json trong thư mục dữ liệu gốc
        json_files = glob.glob(os.path.join(train_data_dir, '*.json'))
        total_files = len(json_files)

        if total_files == 0:
            print(f"LỖI: Không tìm thấy file .json nào trong '{train_data_dir}'.")

        # Lặp qua tất cả các file json gốc
        for file_idx, file_path in enumerate(json_files):
            # Lấy tên động từ từ tên file
            base_name = os.path.basename(file_path)
            verb_name = "_".join(base_name.replace('_train_set.json', '').split('_'))

            print(f"\nĐang tạo index cho động từ {file_idx + 1}/{total_files}: {verb_name}")

            # Đọc dữ liệu từ file JSON gốc
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # Trích xuất các câu từ trường 'text'
            sentences = [item['text'] for item in data if 'text' in item and item['text'].strip()]

            # Chuẩn bị một danh sách để chứa các mục index cho động từ này
            verb_index_list = []

            # Lặp qua từng câu để tạo mục index
            for i, sentence_text in enumerate(sentences):
                # Xây dựng đường dẫn đến file vector .pt tương ứng
                vector_file_name = f"sentence_{i}.pt"
                vector_file_path = os.path.join(vectors_base_dir, verb_name, vector_file_name)

                # ***THAY ĐỔI: Thêm "verb_name" vào mục index***
                index_entry = {
                    "sentence_text": sentence_text,
                    "verb_name": verb_name,
                    "vector_file_path": vector_file_path
                }

                # Thêm mục index vào danh sách
                verb_index_list.append(index_entry)

            # Sau khi xử lý hết các câu, lưu danh sách index vào file .json riêng
            output_index_path = os.path.join(index_output_dir, f"{verb_name}.json")
            with open(output_index_path, 'w', encoding='utf-8') as f:
                json.dump(verb_index_list, f, indent=4, ensure_ascii=False)

            print(f"  -> Đã tạo và lưu file index '{verb_name}.json' với {len(verb_index_list)} mục.")

    except Exception as e:
        print(f"\nĐã xảy ra lỗi không mong muốn: {e}")

    print("\n--- QUÁ TRÌNH TẠO INDEX ĐÃ HOÀN TẤT! ---")


- **Cấu trúc thư mục đầu ra kỳ vọng**
```text
.../train_vectors_pretrained_gramvar/
├── begin_1/
│   ├── sentence_0.pt
│   ├── sentence_1.pt
│   └── ...
├── bind_2/
│   ├── sentence_0.pt
│   └── ...
└── ...

.../verb_indexes_pretrained_gramvar_train/
├── begin_1.json
├── bind_2.json
└── ...

## Trích xuất hidden states theo từng động từ (ParaVE) và lưu thành file `.pt`

- **Mục đích**
  - Lấy biểu diễn ẩn (*hidden states*) của **mọi lớp** trong mô hình BioBERT gốc (`dmis-lab/biobert-base-cased-v1.2`) cho **từng câu** trong tập train của **ParaVE**.  
  - Lưu lại dưới dạng file `.pt` theo **từng động từ** để phục vụ các bước:
    - Tính **mean/std normalization**,  
    - So sánh với biểu diễn từ mô hình fine-tuned,  
    - Huấn luyện mô hình downstream (ví dụ: SRL, argument classification,...).

- **Đầu vào**
  - Thư mục JSON chứa dữ liệu train tách theo động từ:  
    `.../Clean_Dataset/Corpus/Split_ParaVE/Train/*.json`
  - Mỗi file `.json` là danh sách các đối tượng có khóa `text` — chứa câu cần xử lý.

- **Đầu ra**
  - Cấu trúc thư mục vector được tạo tự động:  
    `.../train_vectors_pretrained_parave/<verb_name>/sentence_<i>.pt`
  - Mỗi file `.pt` là **dictionary 13 khóa**:
    - `layer_0, layer_1, ..., layer_12` → `Tensor[num_tokens, 768]`  
      (biểu diễn của từng token ở mỗi lớp trong BioBERT).

- **Quy trình**
  1. **Tải mô hình & tokenizer pre-trained**
     - Model:  
       `AutoModel.from_pretrained(PRETRAINED_MODEL_NAME)` (không có classification head).  
     - Tokenizer:  
       `AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)`
  2. **Đọc dữ liệu** từ từng file JSON trong thư mục `Train/`.
  3. **Lặp từng câu:**
     - Tokenize câu → forward mô hình với `output_hidden_states=True`.  
     - Nhận `outputs.hidden_states` (tuple gồm 13 lớp).  
     - Tạo dict: `{f'layer_{i}': layer_tensor.squeeze(0)}`.  
     - Lưu `sentence_<i>.pt` trong thư mục riêng của từng động từ.
  4. **Lặp cho toàn bộ động từ** đến khi hoàn tất.

In [ ]:
# --- 1. THIẾT LẬP CÁC ĐƯỜNG DẪN VÀ TẢI MÔ HÌNH ---

# Đường dẫn thư mục gốc trên Google Drive
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'

# Đường dẫn đến thư mục chứa các file dữ liệu GỐC
train_data_dir = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Clean_Dataset/Corpus/Split_ParaVE/Train'

# Đường dẫn đến thư mục MỚI để lưu các vector từ mô hình PRE-TRAINED
output_base_dir = os.path.join(drive_base_path, 'Representation Vector/Train/Pretrained/train_vectors_pretrained_parave')
os.makedirs(output_base_dir, exist_ok=True)

# Tải tokenizer và mô hình pre-trained
print("--- Bắt đầu tải mô hình Pre-trained (BioBERT gốc) ---")
try:
    # Tên của mô hình pre-train trên Hugging Face Hub
    PRETRAINED_MODEL_NAME = "dmis-lab/biobert-base-cased-v1.2"

    tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
    # QUAN TRỌNG: Sử dụng AutoModel để tải mô hình gốc, không có lớp phân loại
    model_pt = AutoModel.from_pretrained(PRETRAINED_MODEL_NAME)
    print(" -> Tải thành công tokenizer và mô hình pre-trained!")
except Exception as e:
    print(f"LỖI: Không thể tải mô hình từ Hugging Face. Lỗi: {e}")
    exit()

# --- 2. HÀM XỬ LÝ CỐT LÕI ---

def get_hidden_states_from_pretrained(model, tokenizer, text):
    """
    Trích xuất các trạng thái ẩn (vector biểu diễn) từ mỗi lớp của mô hình pre-trained.
    """
    inputs = tokenizer(text, return_tensors="pt")
    model.eval()
    with torch.no_grad():
        # Yêu cầu mô hình trả về kết quả từ tất cả các lớp
        outputs = model(**inputs, output_hidden_states=True)

    # outputs.hidden_states là một tuple chứa các tensor biểu diễn từ mỗi lớp
    return outputs.hidden_states

# --- 3. THỰC THI CHƯƠNG TRÌNH ---

if __name__ == "__main__":
    print("\n--- Bắt đầu quá trình trích xuất vector từ mô hình Pre-trained ---")

    try:
        # Tìm tất cả các file .json trong thư mục dữ liệu gốc
        json_files = glob.glob(os.path.join(train_data_dir, '*.json'))
        total_files = len(json_files)

        if total_files == 0:
            print(f"LỖI: Không tìm thấy file .json nào trong '{train_data_dir}'.")

        # Lặp qua tất cả các file json gốc
        for file_idx, file_path in enumerate(json_files):
            # Lấy tên động từ từ tên file
            base_name = os.path.basename(file_path)
            verb_name = "_".join(base_name.replace('_train_set.json', '').split('_'))

            print(f"\nĐang xử lý động từ {file_idx + 1}/{total_files}: {verb_name}")

            # Tạo thư mục con tương ứng trong thư mục output
            output_verb_dir = os.path.join(output_base_dir, verb_name)
            os.makedirs(output_verb_dir, exist_ok=True)

            # Đọc dữ liệu từ file JSON gốc
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # Trích xuất các câu từ trường 'text'
            sentences = [item['text'] for item in data if 'text' in item and item['text'].strip()]

            # Lặp qua từng câu
            for i, sentence_text in enumerate(sentences):
                # Trích xuất vector từ mô hình pre-trained
                hidden_states = get_hidden_states_from_pretrained(model_pt, tokenizer, sentence_text)

                # Chuẩn bị dữ liệu để lưu
                vectors_to_save = {}
                for j, layer_rep in enumerate(hidden_states):
                    vectors_to_save[f"layer_{j}"] = layer_rep.squeeze(0)

                # Xây dựng đường dẫn lưu file mới
                file_name = f"sentence_{i}.pt"
                output_path = os.path.join(output_verb_dir, file_name)

                # Lưu file
                torch.save(vectors_to_save, output_path)

            print(f"  -> Đã trích xuất và lưu {len(sentences)} file vector cho động từ '{verb_name}'.")

    except Exception as e:
        print(f"\nĐã xảy ra lỗi không mong muốn: {e}")

    print("\n--- QUÁ TRÌNH TRÍCH XUẤT VECTOR PRE-TRAINED ĐÃ HOÀN TẤT! ---")

- **Lưu ý**
  - Dùng `AutoModel` thay vì `AutoModelForTokenClassification` để trích xuất biểu diễn **thuần gốc** của BioBERT.  
  - Mỗi tensor có `num_tokens` phụ thuộc vào tokenizer (bao gồm `[CLS]`, `[SEP]`).  
  - Nếu có GPU, nên thêm:  
    ```python
    model.to('cuda')
    inputs = {k: v.to('cuda') for k, v in inputs.items()}
    ```  
  - Thư mục đầu ra `train_vectors_pretrained_parave` sẽ được tự động tạo.

- **Kết quả**
  - Cấu trúc đầu ra:
    ```
    .../train_vectors_pretrained_parave/
    ├── block_1/
    │   ├── sentence_0.pt
    │   ├── sentence_1.pt
    │   └── ...
    ├── affect_2/
    │   ├── sentence_0.pt
    │   └── ...
    └── ...
    
  - Mỗi file `.pt` là **dict** gồm 13 lớp:
    ```python
    {
        'layer_0': tensor([...]),  # [num_tokens, 768]
        ...
        'layer_12': tensor([...])
    }
    ```
  - Có thể kiểm tra nhanh:
    ```python
    data = torch.load("train_vectors_pretrained_parave/block_1/sentence_0.pt")
    print(data.keys())            # dict_keys(['layer_0', ..., 'layer_12'])
    print(data['layer_12'].shape) # torch.Size([num_tokens, 768])
    print(data['layer_12'][0][:10])  # 10 giá trị đầu của token [CLS]
    ```

## Tạo file index cho từng động từ (ParaVE)

- **Mục đích**
  - Đọc các file JSON *train* của ParaVE (theo từng động từ) để lấy **câu gốc**.
  - Ghép mỗi câu với **đường dẫn vector `.pt`** đã trích từ BioBERT *pre-trained*.
  - Lưu thành **một file index `.json`/động từ** để tra cứu nhanh ở các bước sau (chuẩn hoá, phân tích, huấn luyện…).

- **Đầu vào**
  - Thư mục câu gốc (theo động từ):  
    `.../Clean_Dataset/Corpus/Split_ParaVE/Train/*.json`
  - Thư mục chứa vector đã trích sẵn (pre-trained):  
    `.../train_vectors_pretrained_parave/<verb_name>/sentence_<i>.pt`

- **Đầu ra**
  - Thư mục index:  
    `.../verb_indexes_pretrained_parave_train/`
  - Mỗi động từ có **1 file**: `<verb_name>.json`, là một **list** các entry:
    ```json
    [
      {
        "sentence_text": "<câu gốc>",
        "verb_name": "<verb_name>",
        "vector_file_path": ".../train_vectors_pretrained_parave/<verb_name>/sentence_<i>.pt"
      },
      ...
    ]
    ```

- **Quy trình**
  1. Duyệt tất cả `*.json` trong `Split_ParaVE/Train/`.
  2. Suy ra `verb_name` từ tên file (ví dụ: `begin_1_train_set.json` → `begin_1`).
  3. Đọc list câu `text`.
  4. Với từng câu **i**:
     - Ghép đường dẫn vector: `train_vectors_pretrained_parave/<verb_name>/sentence_i.pt`.
     - Tạo entry chứa `sentence_text`, `verb_name`, `vector_file_path`.
  5. Ghi **list** entry ra file `verb_indexes_pretrained_parave_train/<verb_name>.json`.


In [ ]:
# Import các thư viện cần thiết
import os
import json
import glob

# --- 1. THIẾT LẬP CÁC ĐƯỜNG DẪN ---

# Đường dẫn thư mục gốc trên Google Drive
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'

# Đường dẫn đến thư mục chứa các file dữ liệu GỐC (nơi lấy câu)
train_data_dir = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Clean_Dataset/Corpus/Split_ParaVE/Train'

# Đường dẫn đến thư mục chứa các file vector .pt ĐÃ ĐƯỢC TẠO từ mô hình pre-trained
vectors_base_dir = os.path.join(drive_base_path, 'Representation Vector/Train/Pretrained/train_vectors_pretrained_parave')

# Đường dẫn đến thư mục MỚI để lưu các file index
index_output_dir = os.path.join(drive_base_path, 'Representation Vector/Train_Index/Pretrained/verb_indexes_pretrained_parave_train')
os.makedirs(index_output_dir, exist_ok=True)


# --- 2. THỰC THI CHƯƠNG TRÌNH TẠO INDEX ---

if __name__ == "__main__":
    print(f"--- Bắt đầu quá trình tạo index cho các vector Pre-trained ---")
    print(f"Đọc dữ liệu gốc từ: {train_data_dir}")
    print(f"Lưu file index vào: {index_output_dir}")

    try:
        # Tìm tất cả các file .json trong thư mục dữ liệu gốc
        json_files = glob.glob(os.path.join(train_data_dir, '*.json'))
        total_files = len(json_files)

        if total_files == 0:
            print(f"LỖI: Không tìm thấy file .json nào trong '{train_data_dir}'.")

        # Lặp qua tất cả các file json gốc
        for file_idx, file_path in enumerate(json_files):
            # Lấy tên động từ từ tên file
            base_name = os.path.basename(file_path)
            verb_name = "_".join(base_name.replace('_train_set.json', '').split('_'))

            print(f"\nĐang tạo index cho động từ {file_idx + 1}/{total_files}: {verb_name}")

            # Đọc dữ liệu từ file JSON gốc
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # Trích xuất các câu từ trường 'text'
            sentences = [item['text'] for item in data if 'text' in item and item['text'].strip()]

            # Chuẩn bị một danh sách để chứa các mục index cho động từ này
            verb_index_list = []

            # Lặp qua từng câu để tạo mục index
            for i, sentence_text in enumerate(sentences):
                # Xây dựng đường dẫn đến file vector .pt tương ứng
                vector_file_name = f"sentence_{i}.pt"
                vector_file_path = os.path.join(vectors_base_dir, verb_name, vector_file_name)

                # ***THAY ĐỔI: Thêm "verb_name" vào mục index***
                index_entry = {
                    "sentence_text": sentence_text,
                    "verb_name": verb_name,
                    "vector_file_path": vector_file_path
                }

                # Thêm mục index vào danh sách
                verb_index_list.append(index_entry)

            # Sau khi xử lý hết các câu, lưu danh sách index vào file .json riêng
            output_index_path = os.path.join(index_output_dir, f"{verb_name}.json")
            with open(output_index_path, 'w', encoding='utf-8') as f:
                json.dump(verb_index_list, f, indent=4, ensure_ascii=False)

            print(f"  -> Đã tạo và lưu file index '{verb_name}.json' với {len(verb_index_list)} mục.")

    except Exception as e:
        print(f"\nĐã xảy ra lỗi không mong muốn: {e}")

    print("\n--- QUÁ TRÌNH TẠO INDEX ĐÃ HOÀN TẤT! ---")


# Test Dataset - Mô hình Fine-tuned

## Trích xuất hidden states theo từng động từ (GramVar) và lưu thành file `.pt`

- **Mục đích**
  - Lấy biểu diễn ẩn (hidden states) của **mọi lớp (13 lớp)** từ mô hình **BioBERT đã fine-tuned** (`biobert-srl-best-model`) cho **từng câu trong tập TEST**.
  - Lưu các vector `.pt` để phục vụ đánh giá và so sánh với các tập train/validation.

- **Đầu vào**
  - Thư mục dữ liệu test (chia theo động từ):  
    `.../Clean_Dataset/Corpus/Split_GramVar/Test/*.json`
  - Mỗi file JSON chứa danh sách các item có khoá `"text"`.

- **Đầu ra**
  - Cấu trúc thư mục lưu vector:
    ```
    .../test_vectors_finetuned_gramvar/<verb_name>/sentence_<i>.pt
    ```
  - Mỗi file `.pt` là **dict** gồm 13 khoá:
    ```
    {
      'layer_0': Tensor[num_tokens, 768],
      'layer_1': Tensor[num_tokens, 768],
      ...
      'layer_12': Tensor[num_tokens, 768]
    }
    ```

- **Quy trình thực hiện**
  1. **Tải mô hình và tokenizer**
     - Tokenizer: `AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.2")`
     - Model: `AutoModelForTokenClassification.from_pretrained(final_model_path)`  
       *(đây là model đã fine-tuned cho Semantic Role Labeling - SRL)*

  2. **Đọc dữ liệu TEST**
     - Mỗi file `.json` tương ứng một **động từ** (ví dụ `abolish_test_set.json` → `abolish`).
     - Lặp qua tất cả câu trong file, lấy nội dung `text`.

  3. **Trích xuất biểu diễn ẩn**
     - Tokenize từng câu → forward model với `output_hidden_states=True`.
     - Lấy `outputs.hidden_states` (tuple gồm 13 lớp).
     - Lưu vào dict `{f'layer_{i}': layer_tensor.squeeze(0)}`.

  4. **Lưu vector ra file `.pt`**
     - Lưu riêng từng câu, đặt tên `sentence_<i>.pt` trong thư mục theo tên động từ.


In [ ]:
# --- 1. THIẾT LẬP CÁC ĐƯỜNG DẪN VÀ TẢI MÔ HÌNH ---

# Đường dẫn thư mục gốc trên Google Drive
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'

# Đường dẫn đến thư mục chứa mô hình fine-tuned
final_model_path = os.path.join(drive_base_path, 'Finetuned_Models/biobert-srl-best-model')

# Đường dẫn đến thư mục chứa các file dữ liệu TEST
test_data_dir = os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_GramVar/Test')

# Đường dẫn đến thư mục MỚI để lưu các vector của tập TEST
output_base_dir = os.path.join(drive_base_path, 'Representation Vector/Test/Finetuned/test_vectors_finetuned_gramvar')
os.makedirs(output_base_dir, exist_ok=True)

# Tải tokenizer và mô hình fine-tuned
print("--- Bắt đầu tải mô hình Fine-tuned ---")
try:
    tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.2")
    model_ft = AutoModelForTokenClassification.from_pretrained(final_model_path)
    print(" -> Tải thành công tokenizer và mô hình fine-tuned!")
except Exception as e:
    print(f"LỖI: Không thể tải mô hình. Lỗi: {e}")
    exit()

# --- 2. HÀM XỬ LÝ CỐT LÕI ---

def get_hidden_states(model, tokenizer, text):
    """
    Trích xuất các trạng thái ẩn (vector biểu diễn) từ mỗi lớp của mô hình.
    """
    inputs = tokenizer(text, return_tensors="pt")
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
    return outputs.hidden_states

# --- 3. THỰC THI CHƯƠNG TRÌNH ---

if __name__ == "__main__":
    print(f"\n--- Bắt đầu quá trình trích xuất vector cho toàn bộ tập TEST ---")
    print(f"Đọc dữ liệu từ: {test_data_dir}")
    print(f"Lưu kết quả vào: {output_base_dir}")

    try:
        # Tìm tất cả các file .json trong thư mục test
        json_files = glob.glob(os.path.join(test_data_dir, '*.json'))
        total_files = len(json_files)

        if total_files == 0:
            print(f"LỖI: Không tìm thấy file .json nào trong '{test_data_dir}'.")

        # Lặp qua tất cả các file json (mỗi file cho một động từ)
        for file_idx, file_path in enumerate(json_files):
            # Lấy tên động từ từ tên file (ví dụ: 'abolish_test_set.json' -> 'abolish')
            base_name = os.path.basename(file_path)
            verb_name = "_".join(base_name.replace('_test_set.json', '').split('_'))

            print(f"\nĐang xử lý động từ {file_idx + 1}/{total_files}: {verb_name}")

            # Tạo thư mục con tương ứng trong thư mục output
            output_verb_dir = os.path.join(output_base_dir, verb_name)
            os.makedirs(output_verb_dir, exist_ok=True)

            # Đọc dữ liệu từ file JSON
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # Trích xuất các câu từ trường 'text'
            sentences = [item['text'] for item in data if 'text' in item and item['text'].strip()]

            # Lặp qua từng câu
            for i, sentence_text in enumerate(sentences):
                # Trích xuất vector từ mô hình fine-tuned
                hidden_states = get_hidden_states(model_ft, tokenizer, sentence_text)

                # Chuẩn bị dữ liệu để lưu
                vectors_to_save = {}
                for j, layer_rep in enumerate(hidden_states):
                    vectors_to_save[f"layer_{j}"] = layer_rep.squeeze(0)

                # Xây dựng đường dẫn lưu file mới
                file_name = f"sentence_{i}.pt"
                output_path = os.path.join(output_verb_dir, file_name)

                # Lưu file
                torch.save(vectors_to_save, output_path)

            print(f"  -> Đã trích xuất và lưu {len(sentences)} file vector cho động từ '{verb_name}'.")

    except Exception as e:
        print(f"\nĐã xảy ra lỗi không mong muốn: {e}")

    print("\n--- QUÁ TRÌNH TRÍCH XUẤT VECTOR TẬP TEST ĐÃ HOÀN TẤT! ---")


--- Bắt đầu tải mô hình Fine-tuned ---


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

 -> Tải thành công tokenizer và mô hình fine-tuned!

--- Bắt đầu quá trình trích xuất vector cho toàn bộ tập TEST ---
Đọc dữ liệu từ: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Clean_Dataset/Corpus/Split_GramVar/Test
Lưu kết quả vào: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/test_vectors_finetuned_gramvar

Đang xử lý động từ 1/35: begin_2
  -> Đã trích xuất và lưu 28 file vector cho động từ 'begin_2'.

Đang xử lý động từ 2/35: begin_1
  -> Đã trích xuất và lưu 36 file vector cho động từ 'begin_1'.

Đang xử lý động từ 3/35: translate_2
  -> Đã trích xuất và lưu 12 file vector cho động từ 'translate_2'.

Đang xử lý động từ 4/35: transform_1
  -> Đã trích xuất và lưu 24 file vector cho động từ 'transform_1'.

Đang xử lý động từ 5/35: delete
  -> Đã trích xuất và lưu 8 file vector cho động từ 'delete'.

Đang xử lý động từ 6/35: confer
  -> Đã trích xuất và lưu 39 file vector cho động từ 'confer'.

Đang xử lý động từ 7/35: splice_2
  -> Đã trích xuất và lư

- **Cấu trúc thư mục mong đợi**
  ```
    .../test_vectors_finetuned_gramvar/
    ├── abolish/
    │   ├── sentence_0.pt
    │   ├── sentence_1.pt
    │   └── ...
    ├── bind/
    │   ├── sentence_0.pt
    │   ├── sentence_1.pt
    │   └── ...
    └── ...
  ```

- **Lưu ý**
  - Mỗi lớp (`layer_k`) có kích thước `[num_tokens, 768]`.
  - Bao gồm các token đặc biệt `[CLS]` và `[SEP]`.
  - Nếu dùng GPU, thêm `.to("cuda")` cho mô hình và input để tăng tốc.
  - Mỗi file `.pt` có thể được nạp lại bằng `torch.load(path)` cho các bước phân tích sau.

## Tạo file index cho từng động từ (GramVar)

- **Mục đích**
  - Tạo **bộ chỉ mục (index)** liên kết giữa:
    - **Câu gốc** (`sentence_text`)
    - **Tên động từ** (`verb_name`)
    - **Đường dẫn đầy đủ tới file vector `.pt`**  
      (đã được trích xuất từ mô hình **BioBERT Fine-tuned**).
  - Giúp các bước sau (như chuẩn hoá, thống kê, hoặc đánh giá) **dễ dàng truy cập** vector tương ứng cho từng câu.


- **Đầu vào**
  - Các file `.json` trong thư mục:
    ```
    .../Clean_Dataset/Corpus/Split_GramVar/Test/*.json
    ```
  - Mỗi file chứa danh sách các câu có khoá `"text"`.

  - Thư mục vector tương ứng (đã được tạo từ mô hình fine-tuned):
    ```
    .../test_vectors_finetuned_gramvar/<verb_name>/sentence_<i>.pt
    ```

- **Đầu ra**
  - Các file index `.json` trong:
    ```
    .../verb_indexes_finetuned_gramvar_test/
    ```
  - Cấu trúc mỗi file:
    ```json
    [
        {
            "sentence_text": "Protein A binds with Protein B.",
            "verb_name": "bind",
            "vector_file_path": "/content/.../test_vectors_finetuned_gramvar/bind/sentence_0.pt"
        },
        {
            "sentence_text": "DNA interacts with RNA polymerase.",
            "verb_name": "interact",
            "vector_file_path": "/content/.../test_vectors_finetuned_gramvar/interact/sentence_1.pt"
        }
    ]
    ```

- **Quy trình**
  1. **Đọc toàn bộ file .json** trong thư mục TEST.
  2. **Trích xuất danh sách câu** từ khoá `"text"`.
  3. **Sinh đường dẫn vector tương ứng** theo mẫu:
     ```
     <vectors_base_dir>/<verb_name>/sentence_<i>.pt
     ```
  4. **Tạo danh sách index** gồm 3 trường:
     - `"sentence_text"`  
     - `"verb_name"`  
     - `"vector_file_path"`
  5. **Lưu danh sách vào file `<verb_name>.json`** trong thư mục index đầu ra.

In [ ]:
# --- 1. THIẾT LẬP CÁC ĐƯỜNG DẪN ---

# Đường dẫn thư mục gốc trên Google Drive
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'

# Đường dẫn đến thư mục chứa các file dữ liệu TEST GỐC (nơi lấy câu)
test_data_dir = os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_GramVar/Test')

# Đường dẫn đến thư mục chứa các file vector .pt ĐÃ ĐƯỢỢC TẠO từ mô hình fine-tuned
vectors_base_dir = os.path.join(drive_base_path, 'test_vectors_finetuned_gramvar')

# Đường dẫn đến thư mục MỚI để lưu các file index cho tập test
index_output_dir = os.path.join(drive_base_path, 'verb_indexes_finetuned_gramvar_test')
os.makedirs(index_output_dir, exist_ok=True)


# --- 2. THỰC THI CHƯƠNG TRÌNH TẠO INDEX ---

if __name__ == "__main__":
    print(f"--- Bắt đầu quá trình tạo index cho các vector Test (Fine-tuned) ---")
    print(f"Đọc dữ liệu gốc từ: {test_data_dir}")
    print(f"Lưu file index vào: {index_output_dir}")

    try:
        # Tìm tất cả các file .json trong thư mục dữ liệu test
        json_files = glob.glob(os.path.join(test_data_dir, '*.json'))
        total_files = len(json_files)

        if total_files == 0:
            print(f"LỖI: Không tìm thấy file .json nào trong '{test_data_dir}'.")

        # Lặp qua tất cả các file json gốc
        for file_idx, file_path in enumerate(json_files):
            # Lấy tên động từ từ tên file (ví dụ: 'abolish_test_set.json' -> 'abolish')
            base_name = os.path.basename(file_path)
            verb_name = "_".join(base_name.replace('_test_set.json', '').split('_'))

            print(f"\nĐang tạo index cho động từ {file_idx + 1}/{total_files}: {verb_name}")

            # Đọc dữ liệu từ file JSON gốc
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # Trích xuất các câu từ trường 'text'
            sentences = [item['text'] for item in data if 'text' in item and item['text'].strip()]

            # Chuẩn bị một danh sách để chứa các mục index cho động từ này
            verb_index_list = []

            # Lặp qua từng câu để tạo mục index
            for i, sentence_text in enumerate(sentences):
                # Xây dựng đường dẫn đến file vector .pt tương ứng
                vector_file_name = f"sentence_{i}.pt"
                vector_file_path = os.path.join(vectors_base_dir, verb_name, vector_file_name)

                # Tạo một mục index với cấu trúc bạn yêu cầu
                index_entry = {
                    "sentence_text": sentence_text,
                    "verb_name": verb_name,
                    "vector_file_path": vector_file_path
                }

                # Thêm mục index vào danh sách
                verb_index_list.append(index_entry)

            # Sau khi xử lý hết các câu, lưu danh sách index vào file .json riêng
            output_index_path = os.path.join(index_output_dir, f"{verb_name}.json")
            with open(output_index_path, 'w', encoding='utf-8') as f:
                json.dump(verb_index_list, f, indent=4, ensure_ascii=False)

            print(f"  -> Đã tạo và lưu file index '{verb_name}.json' với {len(verb_index_list)} mục.")

    except Exception as e:
        print(f"\nĐã xảy ra lỗi không mong muốn: {e}")

    print("\n--- QUÁ TRÌNH TẠO INDEX CHO TẬP TEST ĐÃ HOÀN TẤT! ---")


--- Bắt đầu quá trình tạo index cho các vector Test (Fine-tuned) ---
Đọc dữ liệu gốc từ: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Clean_Dataset/Corpus/Split_GramVar/Test
Lưu file index vào: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/verb_indexes_finetuned_gramvar_test

Đang tạo index cho động từ 1/35: begin_2
  -> Đã tạo và lưu file index 'begin_2.json' với 28 mục.

Đang tạo index cho động từ 2/35: begin_1
  -> Đã tạo và lưu file index 'begin_1.json' với 36 mục.

Đang tạo index cho động từ 3/35: translate_2
  -> Đã tạo và lưu file index 'translate_2.json' với 12 mục.

Đang tạo index cho động từ 4/35: transform_1
  -> Đã tạo và lưu file index 'transform_1.json' với 24 mục.

Đang tạo index cho động từ 5/35: delete
  -> Đã tạo và lưu file index 'delete.json' với 8 mục.

Đang tạo index cho động từ 6/35: confer
  -> Đã tạo và lưu file index 'confer.json' với 39 mục.

Đang tạo index cho động từ 7/35: splice_2
  -> Đã tạo và lưu file index 'splice_2.json' v

- **Cấu trúc thư mục kết quả**
  ```bash
  .../verb_indexes_finetuned_gramvar_test/
  ├── activate.json
  ├── bind.json
  ├── block.json
  └── ...

## Trích xuất hidden states theo từng động từ (ParaVE) và lưu thành file `.pt`

- **Mục đích**
  - Trích xuất **biểu diễn ẩn (hidden states)** của tất cả **13 lớp** từ mô hình BioBERT đã **fine-tuned** cho **từng câu trong tập TEST** (ParaVE).
  - Lưu kết quả ra file `.pt` riêng cho từng câu, phục vụ các bước đánh giá, phân tích hoặc chuẩn hoá sau này.

- **Đầu vào**
  - Thư mục chứa dữ liệu test (chia theo động từ):  
    `.../Clean_Dataset/Corpus/Split_ParaVE/Test/*.json`
  - Mỗi file `.json` là danh sách các item có khoá `"text"`:
    ```json
    [
      {"text": "The protein interacts with the gene."},
      {"text": "They inhibit this pathway strongly."},
      ...
    ]
    ```

- **Đầu ra**
  - Cấu trúc thư mục vector:
    ```
    .../test_vectors_finetuned_parave/<verb_name>/sentence_<i>.pt
    ```
  - Mỗi file `.pt` là một **dict gồm 13 lớp**:
    ```python
    {
      'layer_0': Tensor[num_tokens, 768],
      'layer_1': Tensor[num_tokens, 768],
      ...
      'layer_12': Tensor[num_tokens, 768]
    }
    ```
    > Mỗi `layer_k` là biểu diễn vector của tất cả token trong câu tại lớp tương ứng.


- **Quy trình**
  1. **Tải mô hình & tokenizer**
     - Tokenizer: `AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.2")`
     - Model: `AutoModelForTokenClassification.from_pretrained(final_model_path)`
       - Mô hình `biobert-srl-best-model` đã fine-tuned trên nhiệm vụ **Semantic Role Labeling (SRL)**.
  
  2. **Đọc dữ liệu**
     - Duyệt qua tất cả `.json` trong thư mục TEST.
     - Mỗi file ứng với một động từ (ví dụ: `absorb_test_set.json` → động từ `absorb`).
  
  3. **Trích xuất vector**
     - Tokenize từng câu → forward model với `output_hidden_states=True`.
     - Lấy tuple `hidden_states` chứa 13 lớp (embedding + 12 transformer layers).
     - Lưu từng lớp vào `dict` `{f"layer_{i}": tensor.squeeze(0)}`.
  
  4. **Lưu file vector**
     - Ghi ra `.pt` cho từng câu dưới thư mục riêng tương ứng với động từ.


In [ ]:
# --- 1. THIẾT LẬP CÁC ĐƯỜNG DẪN VÀ TẢI MÔ HÌNH ---

# Đường dẫn thư mục gốc trên Google Drive
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'

# Đường dẫn đến thư mục chứa mô hình fine-tuned
final_model_path = os.path.join(drive_base_path, 'Finetuned_Models/biobert-srl-best-model')

# Đường dẫn đến thư mục chứa các file dữ liệu TEST
test_data_dir = os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_ParaVE/Test')

# Đường dẫn đến thư mục MỚI để lưu các vector của tập TEST
output_base_dir = os.path.join(drive_base_path, 'Representation Vector/Test/Finetuned/test_vectors_finetuned_parave')
os.makedirs(output_base_dir, exist_ok=True)

# Tải tokenizer và mô hình fine-tuned
print("--- Bắt đầu tải mô hình Fine-tuned ---")
try:
    tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.2")
    model_ft = AutoModelForTokenClassification.from_pretrained(final_model_path)
    print(" -> Tải thành công tokenizer và mô hình fine-tuned!")
except Exception as e:
    print(f"LỖI: Không thể tải mô hình. Lỗi: {e}")
    exit()

# --- 2. HÀM XỬ LÝ CỐT LÕI ---

def get_hidden_states(model, tokenizer, text):
    """
    Trích xuất các trạng thái ẩn (vector biểu diễn) từ mỗi lớp của mô hình.
    """
    inputs = tokenizer(text, return_tensors="pt")
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
    return outputs.hidden_states

# --- 3. THỰC THI CHƯƠNG TRÌNH ---

if __name__ == "__main__":
    print(f"\n--- Bắt đầu quá trình trích xuất vector cho toàn bộ tập TEST ---")
    print(f"Đọc dữ liệu từ: {test_data_dir}")
    print(f"Lưu kết quả vào: {output_base_dir}")

    try:
        # Tìm tất cả các file .json trong thư mục test
        json_files = glob.glob(os.path.join(test_data_dir, '*.json'))
        total_files = len(json_files)

        if total_files == 0:
            print(f"LỖI: Không tìm thấy file .json nào trong '{test_data_dir}'.")

        # Lặp qua tất cả các file json (mỗi file cho một động từ)
        for file_idx, file_path in enumerate(json_files):
            # Lấy tên động từ từ tên file (ví dụ: 'abolish_test_set.json' -> 'abolish')
            base_name = os.path.basename(file_path)
            verb_name = "_".join(base_name.replace('_test_set.json', '').split('_'))

            print(f"\nĐang xử lý động từ {file_idx + 1}/{total_files}: {verb_name}")

            # Tạo thư mục con tương ứng trong thư mục output
            output_verb_dir = os.path.join(output_base_dir, verb_name)
            os.makedirs(output_verb_dir, exist_ok=True)

            # Đọc dữ liệu từ file JSON
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # Trích xuất các câu từ trường 'text'
            sentences = [item['text'] for item in data if 'text' in item and item['text'].strip()]

            # Lặp qua từng câu
            for i, sentence_text in enumerate(sentences):
                # Trích xuất vector từ mô hình fine-tuned
                hidden_states = get_hidden_states(model_ft, tokenizer, sentence_text)

                # Chuẩn bị dữ liệu để lưu
                vectors_to_save = {}
                for j, layer_rep in enumerate(hidden_states):
                    vectors_to_save[f"layer_{j}"] = layer_rep.squeeze(0)

                # Xây dựng đường dẫn lưu file mới
                file_name = f"sentence_{i}.pt"
                output_path = os.path.join(output_verb_dir, file_name)

                # Lưu file
                torch.save(vectors_to_save, output_path)

            print(f"  -> Đã trích xuất và lưu {len(sentences)} file vector cho động từ '{verb_name}'.")

    except Exception as e:
        print(f"\nĐã xảy ra lỗi không mong muốn: {e}")

    print("\n--- QUÁ TRÌNH TRÍCH XUẤT VECTOR TẬP TEST ĐÃ HOÀN TẤT! ---")


--- Bắt đầu tải mô hình Fine-tuned ---
 -> Tải thành công tokenizer và mô hình fine-tuned!

--- Bắt đầu quá trình trích xuất vector cho toàn bộ tập TEST ---
Đọc dữ liệu từ: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Clean_Dataset/Corpus/Split_ParaVE/Test
Lưu kết quả vào: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/test_vectors_finetuned_parave

Đang xử lý động từ 1/35: catalyse
  -> Đã trích xuất và lưu 7 file vector cho động từ 'catalyse'.

Đang xử lý động từ 2/35: result
  -> Đã trích xuất và lưu 31 file vector cho động từ 'result'.

Đang xử lý động từ 3/35: eliminate
  -> Đã trích xuất và lưu 5 file vector cho động từ 'eliminate'.

Đang xử lý động từ 4/35: develop
  -> Đã trích xuất và lưu 1 file vector cho động từ 'develop'.

Đang xử lý động từ 5/35: translate_3
  -> Đã trích xuất và lưu 25 file vector cho động từ 'translate_3'.

Đang xử lý động từ 6/35: transform_1
  -> Đã trích xuất và lưu 3 file vector cho động từ 'transform_1'.

Đang xử lý động 

- **Cấu trúc thư mục mong đợi**
  ```bash
  .../test_vectors_finetuned_parave/
  ├── absorb/
  │   ├── sentence_0.pt
  │   ├── sentence_1.pt
  │   └── ...
  ├── block/
  │   ├── sentence_0.pt
  │   ├── sentence_1.pt
  │   └── ...
  └── ...

## Tạo file index cho từng động từ (ParaVE)

- **Mục đích**
  - Sinh ra các file **index `.json`** giúp ánh xạ:
    - `sentence_text` → câu gốc trong file test.
    - `verb_name` → tên động từ (được lấy từ tên file).
    - `vector_file_path` → đường dẫn đầy đủ đến file `.pt` tương ứng (vector của câu đó).
  - Dữ liệu index này được dùng để:
    - Dễ dàng truy cập các vector tương ứng trong bước **chuẩn hoá**, **đánh giá**, hoặc **so sánh** giữa mô hình.


- **Đầu vào**
  - Thư mục chứa dữ liệu test:  
    ```
    .../Clean_Dataset/Corpus/Split_ParaVE/Test/*.json
    ```
  - Thư mục chứa vector của mô hình fine-tuned:  
    ```
    .../test_vectors_finetuned_parave/<verb_name>/sentence_<i>.pt
    ```

- **Đầu ra**
  - Thư mục index mới:  
    ```
    .../verb_indexes_finetuned_parave_test/
    ```
  - Mỗi file `.json` trong thư mục này chứa danh sách index cho từng động từ, ví dụ:
    ```json
    [
        {
            "sentence_text": "Protein A activates gene B in humans.",
            "verb_name": "activate",
            "vector_file_path": "/content/.../test_vectors_finetuned_parave/activate/sentence_0.pt"
        },
        {
            "sentence_text": "This enzyme catalyzes the reaction efficiently.",
            "verb_name": "catalyze",
            "vector_file_path": "/content/.../test_vectors_finetuned_parave/catalyze/sentence_1.pt"
        }
    ]
    ```

- **Quy trình**
  1. **Đọc toàn bộ file JSON test** trong thư mục `Split_ParaVE/Test/`.
  2. **Trích xuất câu** từ trường `"text"`.
  3. **Sinh đường dẫn vector tương ứng** theo mẫu:
     ```
     .../test_vectors_finetuned_parave/<verb_name>/sentence_<i>.pt
     ```
  4. **Tạo danh sách index** gồm:
     - `sentence_text`
     - `verb_name`
     - `vector_file_path`
  5. **Lưu danh sách index** vào file `<verb_name>.json` trong thư mục index đầu ra.

In [ ]:
# --- 1. THIẾT LẬP CÁC ĐƯỜNG DẪN ---

# Đường dẫn thư mục gốc trên Google Drive
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'

# Đường dẫn đến thư mục chứa các file dữ liệu TEST GỐC (nơi lấy câu)
test_data_dir = os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_ParaVE/Test')

# Đường dẫn đến thư mục chứa các file vector .pt ĐÃ ĐƯỢỢC TẠO từ mô hình fine-tuned
vectors_base_dir = os.path.join(drive_base_path, 'Representation Vector/Test/Finetuned/test_vectors_finetuned_parave')

# Đường dẫn đến thư mục MỚI để lưu các file index cho tập test
index_output_dir = os.path.join(drive_base_path, 'Representation Vector/Test_Index/Finetuned/verb_indexes_finetuned_parave_test')
os.makedirs(index_output_dir, exist_ok=True)


# --- 2. THỰC THI CHƯƠNG TRÌNH TẠO INDEX ---

if __name__ == "__main__":
    print(f"--- Bắt đầu quá trình tạo index cho các vector Test (Fine-tuned) ---")
    print(f"Đọc dữ liệu gốc từ: {test_data_dir}")
    print(f"Lưu file index vào: {index_output_dir}")

    try:
        # Tìm tất cả các file .json trong thư mục dữ liệu test
        json_files = glob.glob(os.path.join(test_data_dir, '*.json'))
        total_files = len(json_files)

        if total_files == 0:
            print(f"LỖI: Không tìm thấy file .json nào trong '{test_data_dir}'.")

        # Lặp qua tất cả các file json gốc
        for file_idx, file_path in enumerate(json_files):
            # Lấy tên động từ từ tên file (ví dụ: 'abolish_test_set.json' -> 'abolish')
            base_name = os.path.basename(file_path)
            verb_name = "_".join(base_name.replace('_test_set.json', '').split('_'))

            print(f"\nĐang tạo index cho động từ {file_idx + 1}/{total_files}: {verb_name}")

            # Đọc dữ liệu từ file JSON gốc
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # Trích xuất các câu từ trường 'text'
            sentences = [item['text'] for item in data if 'text' in item and item['text'].strip()]

            # Chuẩn bị một danh sách để chứa các mục index cho động từ này
            verb_index_list = []

            # Lặp qua từng câu để tạo mục index
            for i, sentence_text in enumerate(sentences):
                # Xây dựng đường dẫn đến file vector .pt tương ứng
                vector_file_name = f"sentence_{i}.pt"
                vector_file_path = os.path.join(vectors_base_dir, verb_name, vector_file_name)

                # Tạo một mục index với cấu trúc bạn yêu cầu
                index_entry = {
                    "sentence_text": sentence_text,
                    "verb_name": verb_name,
                    "vector_file_path": vector_file_path
                }

                # Thêm mục index vào danh sách
                verb_index_list.append(index_entry)

            # Sau khi xử lý hết các câu, lưu danh sách index vào file .json riêng
            output_index_path = os.path.join(index_output_dir, f"{verb_name}.json")
            with open(output_index_path, 'w', encoding='utf-8') as f:
                json.dump(verb_index_list, f, indent=4, ensure_ascii=False)

            print(f"  -> Đã tạo và lưu file index '{verb_name}.json' với {len(verb_index_list)} mục.")

    except Exception as e:
        print(f"\nĐã xảy ra lỗi không mong muốn: {e}")

    print("\n--- QUÁ TRÌNH TẠO INDEX CHO TẬP TEST ĐÃ HOÀN TẤT! ---")


--- Bắt đầu quá trình tạo index cho các vector Test (Fine-tuned) ---
Đọc dữ liệu gốc từ: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Clean_Dataset/Corpus/Split_ParaVE/Test
Lưu file index vào: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/verb_indexes_finetuned_parave_test

Đang tạo index cho động từ 1/35: catalyse
  -> Đã tạo và lưu file index 'catalyse.json' với 7 mục.

Đang tạo index cho động từ 2/35: result
  -> Đã tạo và lưu file index 'result.json' với 31 mục.

Đang tạo index cho động từ 3/35: eliminate
  -> Đã tạo và lưu file index 'eliminate.json' với 5 mục.

Đang tạo index cho động từ 4/35: develop
  -> Đã tạo và lưu file index 'develop.json' với 1 mục.

Đang tạo index cho động từ 5/35: translate_3
  -> Đã tạo và lưu file index 'translate_3.json' với 25 mục.

Đang tạo index cho động từ 6/35: transform_1
  -> Đã tạo và lưu file index 'transform_1.json' với 3 mục.

Đang tạo index cho động từ 7/35: begin_2
  -> Đã tạo và lưu file index 'begin_2.json' 

- **Cấu trúc thư mục kết quả**
  ```bash
  .../verb_indexes_finetuned_parave_test/
  ├── activate.json
  ├── bind.json
  ├── block.json
  ├── catalyze.json
  └── ...

# Test Dataset - Mô hình Pre-trained

## Trích xuất hidden states theo từng động từ (GramVar) và lưu thành file `.pt`

- **Mục đích**
  - Lấy **hidden states của 13 lớp** (embedding + 12 transformer layers) từ mô hình **BioBERT gốc** `dmis-lab/biobert-base-cased-v1.2` **không fine-tuned** cho **từng câu** trong tập TEST.
  - Lưu mỗi câu thành một file `.pt` để dùng cho thống kê / chuẩn hoá / so sánh với vector fine-tuned.

- **Đầu vào**
  - Thư mục JSON của tập TEST (chia theo động từ):  
    `.../Clean_Dataset/Corpus/Split_GramVar/Test/*.json`
  - Mỗi file `.json` là một list các object có khóa `"text"`.

- **Đầu ra**
  - Cấu trúc thư mục vector:
    ```
    .../test_vectors_pretrained_gramvar/<verb_name>/sentence_<i>.pt
    ```
  - Mỗi file `.pt` là **dict 13 khóa**:
    ```python
    {
      'layer_0':  Tensor[num_tokens, 768],
      'layer_1':  Tensor[num_tokens, 768],
      ...
      'layer_12': Tensor[num_tokens, 768]
    }
    ```
  - `num_tokens` phụ thuộc tokenizer (bao gồm `[CLS]`, `[SEP]`).

- **Quy trình**
  1. **Tải mô hình & tokenizer PRE-TRAINED**
     - `tok = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.2")`
     - `mdl = AutoModel.from_pretrained("dmis-lab/biobert-base-cased-v1.2")`  
       *(dùng `AutoModel`, **không** có head phân loại).*
  2. **Duyệt dữ liệu TEST**
     - Mỗi file `*_test_set.json` tương ứng một động từ → tạo thư mục `<verb_name>/`.
  3. **Trích xuất hidden states**
     - Tokenize từng câu → `mdl(**inputs, output_hidden_states=True)`.
     - Lấy `outputs.hidden_states` (tuple 13 tensors) → lưu thành dict `{f"layer_{j}": hs.squeeze(0)}`.
  4. **Ghi ra file**
     - Lưu theo tên `sentence_<i>.pt` trong thư mục động từ tương ứng.

In [ ]:
# --- 1. THIẾT LẬP CÁC ĐƯỜNG DẪN VÀ TẢI MÔ HÌNH ---

# Đường dẫn thư mục gốc trên Google Drive
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'

# Đường dẫn đến thư mục chứa các file dữ liệu TEST
test_data_dir = os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_GramVar/Test')

# Đường dẫn đến thư mục MỚI để lưu các vector PRE-TRAINED của tập TEST
output_base_dir = os.path.join(drive_base_path, 'Representation Vector/Test/Pretrained/test_vectors_pretrained_gramvar')
os.makedirs(output_base_dir, exist_ok=True)

# Tải tokenizer và mô hình pre-trained
print("--- Bắt đầu tải mô hình Pre-trained (BioBERT gốc) ---")
try:
    PRETRAINED_MODEL_NAME = "dmis-lab/biobert-base-cased-v1.2"
    tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
    # Tải mô hình gốc, không có lớp phân loại ở trên
    model_pt = AutoModel.from_pretrained(PRETRAINED_MODEL_NAME)
    print(" -> Tải thành công tokenizer và mô hình pre-trained!")
except Exception as e:
    print(f"LỖI: Không thể tải mô hình. Lỗi: {e}")
    exit()

# --- 2. HÀM XỬ LÝ CỐT LÕI ---

def get_hidden_states(model, tokenizer, text):
    """
    Trích xuất các trạng thái ẩn (vector biểu diễn) từ mỗi lớp của mô hình.
    """
    inputs = tokenizer(text, return_tensors="pt")
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
    return outputs.hidden_states

# --- 3. THỰC THI CHƯƠNG TRÌNH ---

if __name__ == "__main__":
    print(f"\n--- Bắt đầu quá trình trích xuất vector PRE-TRAINED cho toàn bộ tập TEST ---")
    print(f"Đọc dữ liệu từ: {test_data_dir}")
    print(f"Lưu kết quả vào: {output_base_dir}")

    try:
        # Tìm tất cả các file .json trong thư mục test
        json_files = glob.glob(os.path.join(test_data_dir, '*.json'))
        total_files = len(json_files)

        if total_files == 0:
            print(f"LỖI: Không tìm thấy file .json nào trong '{test_data_dir}'.")

        # Lặp qua tất cả các file json (mỗi file cho một động từ)
        for file_idx, file_path in enumerate(json_files):
            # Lấy tên động từ từ tên file (ví dụ: 'abolish_test_set.json' -> 'abolish')
            base_name = os.path.basename(file_path)
            verb_name = "_".join(base_name.replace('_test_set.json', '').split('_'))

            print(f"\nĐang xử lý động từ {file_idx + 1}/{total_files}: {verb_name}")

            # Tạo thư mục con tương ứng trong thư mục output
            output_verb_dir = os.path.join(output_base_dir, verb_name)
            os.makedirs(output_verb_dir, exist_ok=True)

            # Đọc dữ liệu từ file JSON
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # Trích xuất các câu từ trường 'text'
            sentences = [item['text'] for item in data if 'text' in item and item['text'].strip()]

            # Lặp qua từng câu
            for i, sentence_text in enumerate(sentences):
                # Trích xuất vector từ mô hình PRE-TRAINED
                hidden_states = get_hidden_states(model_pt, tokenizer, sentence_text)

                # Chuẩn bị dữ liệu để lưu
                vectors_to_save = {}
                for j, layer_rep in enumerate(hidden_states):
                    vectors_to_save[f"layer_{j}"] = layer_rep.squeeze(0)

                # Xây dựng đường dẫn lưu file mới
                file_name = f"sentence_{i}.pt"
                output_path = os.path.join(output_verb_dir, file_name)

                # Lưu file
                torch.save(vectors_to_save, output_path)

            print(f"  -> Đã trích xuất và lưu {len(sentences)} file vector cho động từ '{verb_name}'.")

    except Exception as e:
        print(f"\nĐã xảy ra lỗi không mong muốn: {e}")

    print("\n--- QUÁ TRÌNH TRÍCH XUẤT VECTOR TẬP TEST (PRE-TRAINED) ĐÃ HOÀN TẤT! ---")


--- Bắt đầu tải mô hình Pre-trained (BioBERT gốc) ---


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

 -> Tải thành công tokenizer và mô hình pre-trained!

--- Bắt đầu quá trình trích xuất vector PRE-TRAINED cho toàn bộ tập TEST ---
Đọc dữ liệu từ: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Clean_Dataset/Corpus/Split_GramVar/Test
Lưu kết quả vào: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/test_vectors_pretrained_gramvar

Đang xử lý động từ 1/35: begin_2
  -> Đã trích xuất và lưu 28 file vector cho động từ 'begin_2'.

Đang xử lý động từ 2/35: begin_1
  -> Đã trích xuất và lưu 36 file vector cho động từ 'begin_1'.

Đang xử lý động từ 3/35: translate_2
  -> Đã trích xuất và lưu 12 file vector cho động từ 'translate_2'.

Đang xử lý động từ 4/35: transform_1
  -> Đã trích xuất và lưu 24 file vector cho động từ 'transform_1'.

Đang xử lý động từ 5/35: delete
  -> Đã trích xuất và lưu 8 file vector cho động từ 'delete'.

Đang xử lý động từ 6/35: confer
  -> Đã trích xuất và lưu 39 file vector cho động từ 'confer'.

Đang xử lý động từ 7/35: splice_2
  -> Đã tr

- **Cấu trúc thư mục mong đợi**
  ```bash
  .../test_vectors_pretrained_gramvar/
  ├── abolish/
  │   ├── sentence_0.pt
  │   ├── sentence_1.pt
  │   └── ...
  ├── bind/
  │   ├── sentence_0.pt
  │   ├── sentence_1.pt
  │   └── ...
  └── ...

## Tạo file index cho từng động từ (GramVar)

- **Mục đích**
  - Sinh các file **index `.json`** để ánh xạ:
    - `sentence_text` → câu gốc trong file test.
    - `verb_name` → tên động từ (rút ra từ tên file).
    - `vector_file_path` → đường dẫn đầy đủ đến file vector `.pt` đã trích từ **BioBERT pre-trained**.
  - Bộ index này dùng cho các bước **chuẩn hoá**, **đánh giá**, hoặc **truy vấn nhanh** vector theo câu.

- **Đầu vào**
  - Thư mục JSON của tập test (GramVar):  
    ```
    .../Clean_Dataset/Corpus/Split_GramVar/Test/*.json
    ```
  - Thư mục vector đã trích từ mô hình **pre-trained**:  
    ```
    .../test_vectors_pretrained_gramvar/<verb_name>/sentence_<i>.pt
    ```

- **Đầu ra**
  - Thư mục index mới:  
    ```
    .../verb_indexes_pretrained_gramvar_test/
    ```
  - Mỗi file `<verb_name>.json` là **một danh sách** các entry, ví dụ:
    ```json
    [
      {
        "sentence_text": "Protein A binds to receptor B.",
        "verb_name": "bind",
        "vector_file_path": "/content/.../test_vectors_pretrained_gramvar/bind/sentence_0.pt"
      },
      {
        "sentence_text": "This factor inhibits gene C expression.",
        "verb_name": "inhibit",
        "vector_file_path": "/content/.../test_vectors_pretrained_gramvar/inhibit/sentence_1.pt"
      }
    ]
    ```

- **Quy trình**
  1. **Quét** toàn bộ `*.json` trong `Split_GramVar/Test/`.
  2. **Suy ra** `verb_name` từ tên file (`*_test_set.json` → `<verb_name>`).
  3. **Rút câu** từ trường `"text"` của từng item.
  4. **Ghép đường dẫn** đến vector `.pt` theo mẫu:  
     `test_vectors_pretrained_gramvar/<verb_name>/sentence_<i>.pt`
  5. **Ghi** danh sách entry (list) vào file `.../verb_indexes_pretrained_gramvar_test/<verb_name>.json`.

---

In [ ]:
# --- 1. THIẾT LẬP CÁC ĐƯỜNG DẪN ---

# Đường dẫn thư mục gốc trên Google Drive
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'

# Đường dẫn đến thư mục chứa các file dữ liệu TEST GỐC (nơi lấy câu)
test_data_dir = os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_GramVar/Test')

# Đường dẫn đến thư mục chứa các file vector .pt ĐÃ ĐƯỢC TẠO từ mô hình pre-trained
vectors_base_dir = os.path.join(drive_base_path, 'Representation Vector/Test/Pretrained/test_vectors_pretrained_gramvar')

# Đường dẫn đến thư mục MỚI để lưu các file index cho tập test
index_output_dir = os.path.join(drive_base_path, 'Representation Vector/Test_Index/Pretrained/verb_indexes_pretrained_gramvar_test')
os.makedirs(index_output_dir, exist_ok=True)


# --- 2. THỰC THI CHƯƠNG TRÌNH TẠO INDEX ---

if __name__ == "__main__":
    print(f"--- Bắt đầu quá trình tạo index cho các vector Test (Pre-trained) ---")
    print(f"Đọc dữ liệu gốc từ: {test_data_dir}")
    print(f"Lưu file index vào: {index_output_dir}")

    try:
        # Tìm tất cả các file .json trong thư mục dữ liệu test
        json_files = glob.glob(os.path.join(test_data_dir, '*.json'))
        total_files = len(json_files)

        if total_files == 0:
            print(f"LỖI: Không tìm thấy file .json nào trong '{test_data_dir}'.")

        # Lặp qua tất cả các file json gốc
        for file_idx, file_path in enumerate(json_files):
            # Lấy tên động từ từ tên file (ví dụ: 'abolish_test_set.json' -> 'abolish')
            base_name = os.path.basename(file_path)
            verb_name = "_".join(base_name.replace('_test_set.json', '').split('_'))

            print(f"\nĐang tạo index cho động từ {file_idx + 1}/{total_files}: {verb_name}")

            # Đọc dữ liệu từ file JSON gốc
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # Trích xuất các câu từ trường 'text'
            sentences = [item['text'] for item in data if 'text' in item and item['text'].strip()]

            # Chuẩn bị một danh sách để chứa các mục index cho động từ này
            verb_index_list = []

            # Lặp qua từng câu để tạo mục index
            for i, sentence_text in enumerate(sentences):
                # Xây dựng đường dẫn đến file vector .pt tương ứng
                vector_file_name = f"sentence_{i}.pt"
                vector_file_path = os.path.join(vectors_base_dir, verb_name, vector_file_name)

                # Tạo một mục index với cấu trúc bạn yêu cầu
                index_entry = {
                    "sentence_text": sentence_text,
                    "verb_name": verb_name,
                    "vector_file_path": vector_file_path
                }

                # Thêm mục index vào danh sách
                verb_index_list.append(index_entry)

            # Sau khi xử lý hết các câu, lưu danh sách index vào file .json riêng
            output_index_path = os.path.join(index_output_dir, f"{verb_name}.json")
            with open(output_index_path, 'w', encoding='utf-8') as f:
                json.dump(verb_index_list, f, indent=4, ensure_ascii=False)

            print(f"  -> Đã tạo và lưu file index '{verb_name}.json' với {len(verb_index_list)} mục.")

    except Exception as e:
        print(f"\nĐã xảy ra lỗi không mong muốn: {e}")

    print("\n--- QUÁ TRÌNH TẠO INDEX CHO TẬP TEST (PRE-TRAINED) ĐÃ HOÀN TẤT! ---")


--- Bắt đầu quá trình tạo index cho các vector Test (Pre-trained) ---
Đọc dữ liệu gốc từ: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Clean_Dataset/Corpus/Split_GramVar/Test
Lưu file index vào: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/verb_indexes_pretrained_gramvar_test

Đang tạo index cho động từ 1/35: begin_2
  -> Đã tạo và lưu file index 'begin_2.json' với 28 mục.

Đang tạo index cho động từ 2/35: begin_1
  -> Đã tạo và lưu file index 'begin_1.json' với 36 mục.

Đang tạo index cho động từ 3/35: translate_2
  -> Đã tạo và lưu file index 'translate_2.json' với 12 mục.

Đang tạo index cho động từ 4/35: transform_1
  -> Đã tạo và lưu file index 'transform_1.json' với 24 mục.

Đang tạo index cho động từ 5/35: delete
  -> Đã tạo và lưu file index 'delete.json' với 8 mục.

Đang tạo index cho động từ 6/35: confer
  -> Đã tạo và lưu file index 'confer.json' với 39 mục.

Đang tạo index cho động từ 7/35: splice_2
  -> Đã tạo và lưu file index 'splice_2.json'

- **Cấu trúc thư mục kết quả (minh hoạ)**
  ```bash
  .../verb_indexes_pretrained_gramvar_test/
  ├── bind.json
  ├── inhibit.json
  ├── activate.json
  └── ...

## Trích xuất hidden states theo từng động từ (ParaVE) và lưu thành file `.pt`

- **Mục đích**
  - Lấy **hidden states của 13 lớp** (embedding + 12 transformer layers) từ mô hình **BioBERT gốc** `dmis-lab/biobert-base-cased-v1.2` (không fine-tuned) cho **mỗi câu** trong tập TEST của **ParaVE**.
  - Lưu mỗi câu thành một file `.pt` để phục vụ thống kê/chuẩn hoá/so sánh với vector fine-tuned.


- **Đầu vào**
  - Thư mục JSON của tập TEST (chia theo động từ):  
    `.../Clean_Dataset/Corpus/Split_ParaVE/Test/*.json`
  - Mỗi `.json` là list các object có khoá `text`.


- **Đầu ra**
  - Cấu trúc thư mục:
    ```
    .../test_vectors_pretrained_parave/<verb_name>/sentence_<i>.pt
    ```
  - Mỗi `.pt` là **dict 13 khoá**:
    ```
    {'layer_0': Tensor[num_tokens, 768], ..., 'layer_12': Tensor[num_tokens, 768]}
    ```
  - `num_tokens` phụ thuộc tokenizer (có cả `[CLS]`, `[SEP]`).

- **Quy trình**
  1. **Tải mô hình & tokenizer PRE-TRAINED**
     - `AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.2")`
     - `AutoModel.from_pretrained("dmis-lab/biobert-base-cased-v1.2")` *(không có head phân loại)*.
  2. **Duyệt dữ liệu TEST** theo từng file động từ `*_test_set.json`.
  3. **Với mỗi câu**:
     - Tokenize → `model(**inputs, output_hidden_states=True)`.
     - Lấy `outputs.hidden_states` (tuple 13 tensors).
     - Tạo dict `{f"layer_{j}": hs.squeeze(0)}` và **lưu** `sentence_<i>.pt`.
  4. Lặp đến hết toàn bộ file/câu.

In [ ]:
# --- 1. THIẾT LẬP CÁC ĐƯỜNG DẪN VÀ TẢI MÔ HÌNH ---

# Đường dẫn thư mục gốc trên Google Drive
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'

# Đường dẫn đến thư mục chứa các file dữ liệu TEST
test_data_dir = os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_ParaVE/Test')

# Đường dẫn đến thư mục MỚI để lưu các vector PRE-TRAINED của tập TEST
output_base_dir = os.path.join(drive_base_path, 'Representation Vector/Test/Pretrained/test_vectors_pretrained_parave')
os.makedirs(output_base_dir, exist_ok=True)

# Tải tokenizer và mô hình pre-trained
print("--- Bắt đầu tải mô hình Pre-trained (BioBERT gốc) ---")
try:
    PRETRAINED_MODEL_NAME = "dmis-lab/biobert-base-cased-v1.2"
    tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
    # Tải mô hình gốc, không có lớp phân loại ở trên
    model_pt = AutoModel.from_pretrained(PRETRAINED_MODEL_NAME)
    print(" -> Tải thành công tokenizer và mô hình pre-trained!")
except Exception as e:
    print(f"LỖI: Không thể tải mô hình. Lỗi: {e}")
    exit()

# --- 2. HÀM XỬ LÝ CỐT LÕI ---

def get_hidden_states(model, tokenizer, text):
    """
    Trích xuất các trạng thái ẩn (vector biểu diễn) từ mỗi lớp của mô hình.
    """
    inputs = tokenizer(text, return_tensors="pt")
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
    return outputs.hidden_states

# --- 3. THỰC THI CHƯƠNG TRÌNH ---

if __name__ == "__main__":
    print(f"\n--- Bắt đầu quá trình trích xuất vector PRE-TRAINED cho toàn bộ tập TEST ---")
    print(f"Đọc dữ liệu từ: {test_data_dir}")
    print(f"Lưu kết quả vào: {output_base_dir}")

    try:
        # Tìm tất cả các file .json trong thư mục test
        json_files = glob.glob(os.path.join(test_data_dir, '*.json'))
        total_files = len(json_files)

        if total_files == 0:
            print(f"LỖI: Không tìm thấy file .json nào trong '{test_data_dir}'.")

        # Lặp qua tất cả các file json (mỗi file cho một động từ)
        for file_idx, file_path in enumerate(json_files):
            # Lấy tên động từ từ tên file (ví dụ: 'abolish_test_set.json' -> 'abolish')
            base_name = os.path.basename(file_path)
            verb_name = "_".join(base_name.replace('_test_set.json', '').split('_'))

            print(f"\nĐang xử lý động từ {file_idx + 1}/{total_files}: {verb_name}")

            # Tạo thư mục con tương ứng trong thư mục output
            output_verb_dir = os.path.join(output_base_dir, verb_name)
            os.makedirs(output_verb_dir, exist_ok=True)

            # Đọc dữ liệu từ file JSON
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # Trích xuất các câu từ trường 'text'
            sentences = [item['text'] for item in data if 'text' in item and item['text'].strip()]

            # Lặp qua từng câu
            for i, sentence_text in enumerate(sentences):
                # Trích xuất vector từ mô hình PRE-TRAINED
                hidden_states = get_hidden_states(model_pt, tokenizer, sentence_text)

                # Chuẩn bị dữ liệu để lưu
                vectors_to_save = {}
                for j, layer_rep in enumerate(hidden_states):
                    vectors_to_save[f"layer_{j}"] = layer_rep.squeeze(0)

                # Xây dựng đường dẫn lưu file mới
                file_name = f"sentence_{i}.pt"
                output_path = os.path.join(output_verb_dir, file_name)

                # Lưu file
                torch.save(vectors_to_save, output_path)

            print(f"  -> Đã trích xuất và lưu {len(sentences)} file vector cho động từ '{verb_name}'.")

    except Exception as e:
        print(f"\nĐã xảy ra lỗi không mong muốn: {e}")

    print("\n--- QUÁ TRÌNH TRÍCH XUẤT VECTOR TẬP TEST (PRE-TRAINED) ĐÃ HOÀN TẤT! ---")

--- Bắt đầu tải mô hình Pre-trained (BioBERT gốc) ---
 -> Tải thành công tokenizer và mô hình pre-trained!

--- Bắt đầu quá trình trích xuất vector PRE-TRAINED cho toàn bộ tập TEST ---
Đọc dữ liệu từ: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Clean_Dataset/Corpus/Split_ParaVE/Test
Lưu kết quả vào: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/test_vectors_pretrained_parave

Đang xử lý động từ 1/35: catalyse
  -> Đã trích xuất và lưu 7 file vector cho động từ 'catalyse'.

Đang xử lý động từ 2/35: result
  -> Đã trích xuất và lưu 31 file vector cho động từ 'result'.

Đang xử lý động từ 3/35: eliminate
  -> Đã trích xuất và lưu 5 file vector cho động từ 'eliminate'.

Đang xử lý động từ 4/35: develop
  -> Đã trích xuất và lưu 1 file vector cho động từ 'develop'.

Đang xử lý động từ 5/35: translate_3
  -> Đã trích xuất và lưu 25 file vector cho động từ 'translate_3'.

Đang xử lý động từ 6/35: transform_1
  -> Đã trích xuất và lưu 3 file vector cho động từ 'tr

- **Cấu trúc thư mục mong đợi**
  ```bash
  .../test_vectors_pretrained_parave/
  ├── begin_1/
  │   ├── sentence_0.pt
  │   ├── sentence_1.pt
  │   └── ...
  ├── block/
  │   ├── sentence_0.pt
  │   ├── sentence_1.pt
  │   └── ...
  └── ...

## Tạo file index cho từng động từ (ParaVE)

- **Mục đích**
  - Sinh các file **index `.json`** để ánh xạ mỗi câu test tới đường dẫn vector `.pt` đã trích từ **BioBERT pre-trained**.
  - Phục vụ các bước **chuẩn hoá**, **đánh giá** và **truy vấn** vector theo câu/động từ.


- **Đầu vào**
  - Thư mục JSON của tập test (ParaVE):  
    `.../Clean_Dataset/Corpus/Split_ParaVE/Test/*.json`
  - Thư mục vector **pre-trained** (đã trích sẵn):  
    `.../test_vectors_pretrained_parave/<verb_name>/sentence_<i>.pt`


- **Đầu ra**
  - Thư mục index mới:  
    `.../verb_indexes_pretrained_parave_test/`
  - Mỗi file `<verb_name>.json` là **một danh sách** các entry:
    ```json
    [
      {
        "sentence_text": "Protein A interacts with protein B.",
        "verb_name": "interact",
        "vector_file_path": "/content/.../test_vectors_pretrained_parave/interact/sentence_0.pt"
      },
      {
        "sentence_text": "Complex C binds to DNA.",
        "verb_name": "bind",
        "vector_file_path": "/content/.../test_vectors_pretrained_parave/bind/sentence_1.pt"
      }
    ]
    ```

- **Quy trình**
  1. **Quét** tất cả `*.json` trong `Split_ParaVE/Test/`.
  2. **Suy ra** `verb_name` từ tên file (`*_test_set.json` ➜ `<verb_name>`).
  3. **Rút câu** từ trường `"text"` (bỏ dòng rỗng).
  4. **Ghép đường dẫn** đến vector `.pt` theo mẫu:  
     `test_vectors_pretrained_parave/<verb_name>/sentence_<i>.pt`
  5. **Ghi** danh sách entry (list) vào `verb_indexes_pretrained_parave_test/<verb_name>.json`.

In [ ]:
# --- 1. THIẾT LẬP CÁC ĐƯỜNG DẪN ---

# Đường dẫn thư mục gốc trên Google Drive
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'

# Đường dẫn đến thư mục chứa các file dữ liệu TEST GỐC (nơi lấy câu)
test_data_dir = os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_ParaVE/Test')

# Đường dẫn đến thư mục chứa các file vector .pt ĐÃ ĐƯỢC TẠO từ mô hình pre-trained
vectors_base_dir = os.path.join(drive_base_path, 'Representation Vector/Test/Pretrained/test_vectors_pretrained_parave')

# Đường dẫn đến thư mục MỚI để lưu các file index cho tập test
index_output_dir = os.path.join(drive_base_path, 'Representation Vector/Test_Index/Pretrained/verb_indexes_pretrained_parave_test')
os.makedirs(index_output_dir, exist_ok=True)


# --- 2. THỰC THI CHƯƠNG TRÌNH TẠO INDEX ---

if __name__ == "__main__":
    print(f"--- Bắt đầu quá trình tạo index cho các vector Test (Pre-trained) ---")
    print(f"Đọc dữ liệu gốc từ: {test_data_dir}")
    print(f"Lưu file index vào: {index_output_dir}")

    try:
        # Tìm tất cả các file .json trong thư mục dữ liệu test
        json_files = glob.glob(os.path.join(test_data_dir, '*.json'))
        total_files = len(json_files)

        if total_files == 0:
            print(f"LỖI: Không tìm thấy file .json nào trong '{test_data_dir}'.")

        # Lặp qua tất cả các file json gốc
        for file_idx, file_path in enumerate(json_files):
            # Lấy tên động từ từ tên file (ví dụ: 'abolish_test_set.json' -> 'abolish')
            base_name = os.path.basename(file_path)
            verb_name = "_".join(base_name.replace('_test_set.json', '').split('_'))

            print(f"\nĐang tạo index cho động từ {file_idx + 1}/{total_files}: {verb_name}")

            # Đọc dữ liệu từ file JSON gốc
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # Trích xuất các câu từ trường 'text'
            sentences = [item['text'] for item in data if 'text' in item and item['text'].strip()]

            # Chuẩn bị một danh sách để chứa các mục index cho động từ này
            verb_index_list = []

            # Lặp qua từng câu để tạo mục index
            for i, sentence_text in enumerate(sentences):
                # Xây dựng đường dẫn đến file vector .pt tương ứng
                vector_file_name = f"sentence_{i}.pt"
                vector_file_path = os.path.join(vectors_base_dir, verb_name, vector_file_name)

                # Tạo một mục index với cấu trúc bạn yêu cầu
                index_entry = {
                    "sentence_text": sentence_text,
                    "verb_name": verb_name,
                    "vector_file_path": vector_file_path
                }

                # Thêm mục index vào danh sách
                verb_index_list.append(index_entry)

            # Sau khi xử lý hết các câu, lưu danh sách index vào file .json riêng
            output_index_path = os.path.join(index_output_dir, f"{verb_name}.json")
            with open(output_index_path, 'w', encoding='utf-8') as f:
                json.dump(verb_index_list, f, indent=4, ensure_ascii=False)

            print(f"  -> Đã tạo và lưu file index '{verb_name}.json' với {len(verb_index_list)} mục.")

    except Exception as e:
        print(f"\nĐã xảy ra lỗi không mong muốn: {e}")

    print("\n--- QUÁ TRÌNH TẠO INDEX CHO TẬP TEST (PRE-TRAINED) ĐÃ HOÀN TẤT! ---")


--- Bắt đầu quá trình tạo index cho các vector Test (Pre-trained) ---
Đọc dữ liệu gốc từ: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Clean_Dataset/Corpus/Split_ParaVE/Test
Lưu file index vào: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/verb_indexes_pretrained_parave_test

Đang tạo index cho động từ 1/35: catalyse
  -> Đã tạo và lưu file index 'catalyse.json' với 7 mục.

Đang tạo index cho động từ 2/35: result
  -> Đã tạo và lưu file index 'result.json' với 31 mục.

Đang tạo index cho động từ 3/35: eliminate
  -> Đã tạo và lưu file index 'eliminate.json' với 5 mục.

Đang tạo index cho động từ 4/35: develop
  -> Đã tạo và lưu file index 'develop.json' với 1 mục.

Đang tạo index cho động từ 5/35: translate_3
  -> Đã tạo và lưu file index 'translate_3.json' với 25 mục.

Đang tạo index cho động từ 6/35: transform_1
  -> Đã tạo và lưu file index 'transform_1.json' với 3 mục.

Đang tạo index cho động từ 7/35: begin_2
  -> Đã tạo và lưu file index 'begin_2.json

- **Cấu trúc thư mục (minh hoạ)**
  ```bash
  .../verb_indexes_pretrained_parave_test/
  ├── interact.json
  ├── bind.json
  ├── regulate.json
  └── ...

# Tải vector theo động từ & phân tích chi tiết một câu

- **Mục đích**:  
  - Đọc file **index** theo từng động từ để tìm đúng đường dẫn tới file vector `.pt` của một câu cụ thể.  
  - Tải lại vector biểu diễn (hidden states) đã lưu cho câu đó và **khám phá cấu trúc từng lớp** của mô hình.  

- **Quy trình**

1. **Thiết lập đường dẫn**  
     - `drive_base_path`: thư mục gốc làm việc trên Google Drive.  
     - `index_dir`: nơi chứa các file index JSON (mỗi file tương ứng một động từ, ví dụ `block.json`).  
     - (Tùy chọn) `vectors_dir`: thư mục chứa các vector `.pt` (thường không cần vì đường dẫn tuyệt đối đã có trong file index).  

In [ ]:
# Đường dẫn thư mục gốc trên Google Drive để làm việc
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'

# Đường dẫn đến thư mục chứa các file index .json (mỗi file cho một động từ)
index_dir = os.path.join(drive_base_path, 'Representation Vector/Train_Index/Finetuned/verb_indexes_finetuned_parave_train')

# (Không bắt buộc, vì đường dẫn đầy đủ đã có trong file index)
# Đường dẫn đến thư mục cha chứa các vector đã được xử lý
vectors_dir = os.path.join(drive_base_path, 'Representation Vector/Train/Finetuned/train_vectors_finetuned_parave')

2. **Hàm `load_vector_data(verb, sentence_index)`**  
     - **Input**:  
       - `verb`: tên động từ (ví dụ: `"block"` hoặc `"begin_1"`).  
       - `sentence_index`: chỉ số câu muốn lấy (bắt đầu từ 0).  
     - **Xử lý**:  
       - Đọc file index `{verb}.json` trong `index_dir`.  
       - Lấy ra `sentence_text` (câu gốc) và `vector_file_path` (đường dẫn file `.pt`).  
       - Dùng `torch.load()` để tải `vector_data` (dict: `layer_0`, `layer_1`, …).  
     - **Output**: `tuple (vector_data, sentence_text)` hoặc `(None, None)` nếu lỗi/không tìm thấy.  
     - **An toàn**: Có kiểm tra phạm vi `sentence_index` và bắt lỗi đọc file/json.  

In [ ]:
def load_vector_data(verb, sentence_index):
    """
    Tải dữ liệu vector cho một câu cụ thể dựa vào tên động từ và chỉ số câu.

    Args:
        verb (str): Tên của động từ (ví dụ: 'begin_1').
        sentence_index (int): Chỉ số của câu trong danh sách của động từ đó (bắt đầu từ 0).

    Returns:
        tuple: Một tuple chứa (dữ liệu vector đã tải, câu gốc tương ứng)
               hoặc (None, None) nếu không tìm thấy file.
    """
    try:
        # Xây dựng đường dẫn đến file index .json của động từ
        index_file_path = os.path.join(index_dir, f"{verb}.json")

        # Đọc file index
        with open(index_file_path, 'r', encoding='utf-8') as f:
            index_data = json.load(f) # Đây là một list các dictionary

        # Kiểm tra xem chỉ số câu có hợp lệ không
        if sentence_index >= len(index_data):
            print(f"LỖI: Chỉ số câu {sentence_index} vượt quá số lượng câu ({len(index_data)}) của động từ '{verb}'.")
            return None, None

        # Lấy thông tin của câu cần tìm
        sentence_info = index_data[sentence_index]
        sentence_text = sentence_info['sentence_text']
        vector_file_path = sentence_info['vector_file_path']

        # Tải dữ liệu vector từ file .pt
        print(f"Đang tải file vector từ: {vector_file_path}")
        vector_data = torch.load(vector_file_path)

        return vector_data, sentence_text

    except FileNotFoundError:
        print(f"LỖI: Không tìm thấy file index '{verb}.json' hoặc file vector tương ứng cho câu {sentence_index}.")
        return None, None
    except Exception as e:
        print(f"Đã xảy ra lỗi không mong muốn khi đọc câu {sentence_index} của động từ {verb}: {e}")
        return None, None

 3. **Ví dụ sử dụng (main)**  
     - Chọn mục tiêu:  
       - `target_verb = "block"`  
       - `target_sentence_index = 1`  
     - Gọi `load_vector_data`. Nếu thành công:  
       - In **câu gốc**.  
       - Duyệt qua từng lớp (`layer_0`, `layer_1`, …) và in:  
         - **Kích thước tensor** của lớp (thường là `[num_tokens, 768]` với BioBERT base).  
         - **10 giá trị đầu** của vector token đầu tiên `[CLS]` ở mỗi lớp (minh họa).  

In [ ]:
if __name__ == "__main__":
    # THAY ĐỔI CÁC THAM SỐ NÀY ĐỂ CHỌN CÂU BẠN MUỐN PHÂN TÍCH
    target_verb = "block"
    target_sentence_index = 1 # Phân tích câu đầu tiên (chỉ số 0)

    # Gọi hàm để tải dữ liệu
    loaded_data, original_sentence = load_vector_data(target_verb, target_sentence_index)

    # Kiểm tra và hiển thị kết quả chi tiết
    if loaded_data:
        print("\n--- Phân tích chi tiết các lớp cho câu đã chọn ---")
        print(f"Câu gốc: '{original_sentence}'")

        # 'loaded_data' là một dictionary. Lặp qua từng lớp để in thông tin
        for layer_name, layer_tensor in loaded_data.items():
            # In tên lớp một cách trang trọng (ví dụ: "Layer 0", "Layer 1")
            print(f"\n  --- {layer_name.replace('_', ' ').title()} ---")

            # In kích thước của toàn bộ tensor ở lớp này
            # Kích thước sẽ là [số_lượng_token, 768]
            print(f"  Kích thước tensor: {layer_tensor.shape}")

            # Lấy vector của token đầu tiên ('[CLS]') ở lớp hiện tại để xem ví dụ
            if layer_tensor.shape[0] > 0:
                cls_token_vector = layer_tensor[0]
                print(f"  Vector của token '[CLS]' (10 giá trị đầu):")
                print(f"  {cls_token_vector[:10]}")

Đang tải file vector từ: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/train_vectors_finetuned_parave/block/sentence_1.pt

--- Phân tích chi tiết các lớp cho câu đã chọn ---
Câu gốc: 'The antibody increasing GR by 14-3-3eta for the phosphopeptide Y553 was specific, as inhibition of translation would have blocked labeling.'

  --- Layer 0 ---
  Kích thước tensor: torch.Size([40, 768])
  Vector của token '[CLS]' (10 giá trị đầu):
  tensor([ 0.4710,  0.0489, -0.1954,  0.3613, -0.0913, -0.3245,  0.9731, -0.9639,
        -0.4767, -0.2092])

  --- Layer 1 ---
  Kích thước tensor: torch.Size([40, 768])
  Vector của token '[CLS]' (10 giá trị đầu):
  tensor([ 0.3631,  0.0481, -0.1143,  0.1713, -0.1925, -0.1522,  0.4374, -0.1837,
        -0.2713, -0.0470])

  --- Layer 2 ---
  Kích thước tensor: torch.Size([40, 768])
  Vector của token '[CLS]' (10 giá trị đầu):
  tensor([ 0.3720,  0.0295, -0.0118,  0.0461, -0.2433, -0.2570,  0.5109, -0.4854,
        -0.2953, -0.2195])

  --- Layer 

- **Kết quả**:  
  - Xem được nội dung câu gốc và cấu trúc vector ở **từng lớp** của mô hình cho câu đã chọn.  
  - Xác minh nhanh việc lưu/tải vector `.pt` hoạt động đúng; thuận tiện cho các phân tích tiếp theo (so sánh lớp, tính similarity, trực quan hóa…).  

- **Lưu ý**:  
  - Với BioBERT base, kích thước ẩn là **768**; số lớp thường là **13** tensor nếu bao gồm cả embedding (`layer_0`) + 12 lớp Transformer (`layer_1` … `layer_12`).  
  - Hãy đảm bảo file `{verb}.json` tồn tại trong `index_dir` và đường dẫn `vector_file_path` bên trong là hợp lệ.  